In [ ]:
%matplotlib inline
import time
import numpy as np
from scipy import optimize
from scipy.interpolate import interp1d
from scipy.signal import find_peaks
from scipy.interpolate import *
from scipy.fftpack import fft
import matplotlib.pyplot as plt
import pandas as pd
from pprint import pprint
import os, sys
from time import sleep, monotonic
from scipy.ndimage import gaussian_filter

import qcodes as qc
from qcodes import Parameter
from qcodes import initialise_or_create_database_at
from qcodes import load_or_create_experiment
from qcodes.dataset.plotting import plot_dataset, plot_by_id
from qcodes.utils.metadata import diff_param_values
from qcodes.instrument.specialized_parameters import ElapsedTimeParameter

sys.path.append('M:\\tnw\\ns\\qt\\2D Topo\\code\\qcodes')

In [ ]:
%matplotlib widget

In [ ]:
import matplotlib
import matplotlib.ticker as ticker

In [ ]:
matplotlib.rcParams.update({'font.size': 16})
matplotlib.rc('xtick', labelsize=15) 
matplotlib.rc('ytick', labelsize=15) 

# Smooth dirivative

In [ ]:
def smooth_dir(parametr, N_points = 5):
    
    #parametr => current_DC
    #Ny => Ngate
    #Nx => Nvolt
    
    Ny = parametr.shape[0]
    Nx = parametr.shape[1]
    
    d_parameter = np.zeros_like(parametr)
    
    for y_idx in range(Ny): #for each gate value
        for x_idx in range (Nx): #for each voltage point
            vals_to_fit = parametr[y_idx, max(0,x_idx-N_points):min(Nx, x_idx+N_points)] #select current values to fit
            x_axes = np.arange(vals_to_fit.size) #array from 0 to 2*Npoints+1 of integers (fitting length)
            fit_vals = np.polyfit(x_axes, vals_to_fit, 1) #directly use x_axes, because we take derivative with step
            d_parameter[y_idx, x_idx] = fit_vals[0] #derivative with respect to index
            
    return d_parameter

In [ ]:
def smooth_dir_1D(parametr, N_points = 5):
    
    #parametr => current_DC
    #Ny => Ngate
    #Nx => Nvolt
    
#     Ny = parametr.shape[0]
    Nx = parametr.size
    
    d_parameter = np.zeros_like(parametr)
    for x_idx in range (Nx): #for each voltage point
        vals_to_fit = parametr[max(0,x_idx-N_points):min(Nx, x_idx+N_points)] #select current values to fit
        x_axes = np.arange(vals_to_fit.size) #array from 0 to 2*Npoints+1 of integers (fitting length)
        fit_vals = np.polyfit(x_axes, vals_to_fit, 1) #directly use x_axes, because we take derivative with step
        d_parameter[x_idx] = fit_vals[0] #derivative with respect to index
            
    return d_parameter

# Ivan JJ

In [ ]:
# qc.experiments()

In [ ]:
dataset = qc.load_by_id(23)
print(dataset.exp_name)
cmap = plt.get_cmap('hot')
axes = plot_dataset(dataset, cmap = cmap)
axes
#plt.savefig('../plots/TL2_JJL_IV_1.png')

## Get R from slop

In [ ]:
dataset = qc.load_by_id(32)
voltage = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']
fit_values = np.polyfit(current,voltage,1)
print(np.abs(fit_values[0]))

In [ ]:
dataset = qc.load_by_id(9)
dataset.get_parameter_data()['meas_voltage_K1'].keys()

## IV vs time

In [ ]:
dataset = qc.load_by_id(9)
voltage = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']
time = dataset.get_parameter_data()['meas_voltage_K1']['time']

N_time = np.size(np.unique(time))
N_current = np.size(np.unique(current))

if np.size(voltage) < N_time * N_current:
    print('Interupted scan! Last sweep is disregarded')
    N_time = N_time - 1
    voltage = voltage[0:N_time * N_current]
    current = current[0:N_time * N_current]
    time = time[0:N_time * N_current]
    

voltage = -1*voltage.reshape(N_time, N_current)
current = current.reshape(N_time, N_current)
time = time.reshape(N_time, N_current)

dif_voltage = np.diff(voltage, axis = 1)
dif_current = np.diff(current, axis = 1)
R = dif_voltage/dif_current
difdif_voltage = np.diff(dif_voltage, axis = 1)
R_filt = gaussian_filter( R, sigma=(0.1,1))

In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap('hot')
im = ax.pcolormesh(time/60, 1e9 * current, R_filt, cmap = cmap, vmin = 0)
fig.colorbar(im,ax=ax)
#ax.set_ylim(-100, 100)
#ax.set_xlim(-2, 2)
ax.set_ylabel(r'Current (nA)')
ax.set_xlabel(r'Time (min)')
#plt.savefig('../plots/R(I,B)_J1.png')
plt.show()

## IV vs gate

In [ ]:
dataset = qc.load_by_id(28)
voltage = dataset.get_parameter_data()['meas_voltage_K2']['meas_voltage_K2']
current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']
v_gate = dataset.get_parameter_data()['meas_voltage_K1']['appl_TG']

N_gate = np.size(np.unique(v_gate))
N_current = np.size(np.unique(current))

if np.size(voltage) < N_gate * N_current:
    N_gate = N_gate - 1
    voltage = voltage[0:N_gate * N_current]
    current = current[0:N_gate * N_current]
    v_gate = v_gate[0:N_gate * N_current]
    

voltage = voltage.reshape(N_gate, N_current)
current = current.reshape(N_gate, N_current)
v_gate = v_gate.reshape(N_gate, N_current)

dif_voltage = np.diff(voltage, axis = 1)
dif_current = np.diff(current, axis = 1)
R = dif_voltage/dif_current
difdif_voltage = np.diff(dif_voltage, axis = 1)
R_filt = gaussian_filter( R, sigma=(0.1,2))

I_step = current[0,1] - current[0,0]
R_sm = smooth_dir(voltage, N_points=6) / I_step #scale the derivative

In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap('hot')
im = ax.pcolormesh(v_gate, 1e9 * current, R_sm, cmap = cmap, vmin=0,vmax=600)
cbar = fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)
# ax.set_xlim(-10, 10)
ax.set_ylabel(r'Current (nA)')
ax.set_xlabel(r'V gate (V)')
cbar.set_label(r'Resistance ($\Omega$)', rotation=90)
plt.title('JJ1_JJ_2')
plt.savefig('../plots/R(I,Vg)_JJ1_JJ_2.png')
plt.show()

## IV vs B

In [ ]:
dataset = qc.load_by_id(25)
dataset.get_parameter_data()['meas_voltage_K2'].keys()

In [ ]:
dataset = qc.load_by_id(27)
voltage = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']
field = dataset.get_parameter_data()['meas_voltage_K1']['cryomag_A_field']

N_field = np.size(np.unique(field))
N_current = np.size(np.unique(current))

if np.size(voltage) < N_field * N_current:
    N_field = N_field - 1
    voltage = voltage[0:N_field * N_current]
    current = current[0:N_field * N_current]
    field = field[0:N_field * N_current]
    

voltage = voltage.reshape(N_field, N_current)
current = current.reshape(N_field, N_current)
field = field.reshape(N_field, N_current)

dif_voltage = np.diff(voltage, axis = 1)
dif_current = np.diff(current, axis = 1)
R = dif_voltage/dif_current
difdif_voltage = np.diff(dif_voltage, axis = 1)
R_filt = gaussian_filter( R, sigma=(0,1))

I_step = current[0,1] - current[0,0]
R_sm = smooth_dir(voltage, N_points=6) / I_step #scale the derivative

In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap('hot')

pixel_width = field[1,0]-field[0,0]
pixel_height = current[0,1]-current[0,0]

im = ax.pcolormesh(1e3*(field-pixel_width/2), 1e9 * (current-pixel_height/2), 
                   R_sm, cmap = cmap, vmin=0,vmax=500)
fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)\
# ax.set_xlim(-2, 2)
ax.set_ylabel(r'Current (nA)')
ax.set_xlabel(r'Field (mT)')
plt.title('JJ1_JJ_1')
plt.savefig('../plots/R(I,B)_JJ1_JJ_1_b.png')
plt.show()

## Finding Ic and offset

In [ ]:
difdif_voltage = gaussian_filter( np.diff( gaussian_filter(R_filt, sigma=(0.1,1)), axis = 1), sigma=(0.1,2))

#shape matching due to derivative
field_cut = 1e3 * field[:,0:-2]
current_cut = 1e9 * current[:,0:-2]


#row and coulumn masks
mask_B =  (( field_cut > -1 ) & ( field_cut < 0.5 ) )[:,1]
mask_I = (( current_cut > -300 ) & ( current_cut < 10 ) )[1,:]

field_masked = field_cut[:,mask_I]
field_masked = field_masked[mask_B,:]

current_masked = current_cut[:,mask_I]
current_masked = current_masked[mask_B,:]

difdif_voltage_masked = difdif_voltage[:,mask_I]
difdif_voltage_masked = difdif_voltage_masked[mask_B,:]

R_filt_cut = R_filt[:,0:-1]
R_filt_masked = R_filt_cut[:,mask_I]
R_filt_masked = R_filt_masked[mask_B,:]

### Peaks extraction in $\frac{d^2 V}{dI^2}$

In [ ]:
B_vals = field_masked[:,1]
I_vals = current_masked[1,:]
peak_positions = np.zeros_like(B_vals)

for B_idx, B in enumerate(B_vals):
    d2V_dI2 = -1*difdif_voltage_masked[B_idx, 10:-1]
    I_vals_cut = I_vals[10:-1]
    prominance = 1000
    peaks = find_peaks(d2V_dI2, prominence=prominance)
    while np.size(peaks[0])==0:
#         print('no peaks')
        prominance -=2
        peaks = find_peaks(d2V_dI2, prominence=prominance)
#     print(peaks[0])
    
#     if np.size(peaks[0]) > 1:
#         fig, ax = plt.subplots()
#         ax.plot(I_vals_cut, d2V_dI2)
#         ax.scatter(I_vals_cut[peaks[0]], d2V_dI2[peaks[0]])
#         plt.show()
#         break
        
    if np.size(peaks[0]) > 1:
        peak_positions[B_idx] =  I_vals_cut[peaks[0][np.argmax(peaks[1]['prominences'])]]
    else:
        peak_positions[B_idx] =  I_vals_cut[peaks[0]] if len(peaks[0])>0 else 0
        
#     if np.abs(peak_positions[B_idx]) > 200:
#         fig, ax = plt.subplots()
#         ax.plot(I_vals_cut, d2V_dI2)
#         ax.scatter(I_vals_cut[peaks[0]], d2V_dI2[peaks[0]])
#         plt.show()
#         break
        
    
#     fig, ax = plt.subplots()
#     ax.plot(I_vals_cut, d2V_dI2)
#     ax.scatter(I_vals_cut[peaks[0]], d2V_dI2[peaks[0]])
#     plt.show()
#     break

In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap('hot')
pixel_width = field_masked[1,0]-field_masked[0,0]
pixel_height = current_masked[0,1]-current_masked[0,0]

im = ax.pcolor(field_masked - pixel_width/2, current_masked - pixel_height/2, R_filt_masked, cmap = cmap, vmin= 0, vmax = 300)
fig.colorbar(im,ax=ax)
ax.scatter(B_vals, peak_positions)
ax.set_ylim(-250, 0)
#ax.set_xlim(-0.6, -0.1)
ax.set_ylabel(r'Current (nA)')
ax.set_xlabel(r'Field (mT)')
ax.axvline(-0.3)
plt.show()

In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap('hot')

pixel_width = field[1,0]-field[0,0]
pixel_height = current[0,1]-current[0,0]

im = ax.pcolormesh(1e3*(field-pixel_width/2), 1e9 * (current-pixel_height/2), 
                   R_filt, cmap = cmap, vmin=0,vmax=500)
fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)\
# ax.set_xlim(-2, 2)

ax.plot(B_vals, peak_positions, linewidth = 3, color = 'magenta')

ax.set_ylabel(r'Current (nA)')
ax.set_xlabel(r'Field (mT)')
plt.title('BL3_JJ_2')
# plt.savefig('../plots/R(I,B)_BL3_JJ_2.png')
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(B_vals, -peak_positions, linewidth = 4, color = 'magenta')


plt.show()

## MAR data

In [ ]:
dataset = qc.load_by_id(36)
voltage = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']

lockin = dataset.get_parameter_data()['meas_voltage_R_Lockin1']['meas_voltage_R_Lockin1']
current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']

R_AC = lockin/current_AC

In [ ]:
fig, ax = plt.subplots(constrained_layout = True)
# ax.plot(1e3*voltage[0:-1], R_filt)
# ax.plot(1e3*voltage, R_filt)
ax.plot(1e3*voltage, gaussian_filter(R_AC, sigma=(1)))

ax.set_ylim(200, 280)
# ax.set_ylim(210, 250)
# ax.set_xlim(-2, 2)
ax.set_ylabel(r'$\frac{d \: \mathrm{V}}{d \: \mathrm{I}}$ ($\Omega$)', fontsize = 15)
ax.set_xlabel(r'Voltage (mV)', fontsize = 12)
plt.title('JJ1_JJ_3, $\mathrm{I}_\mathrm{AC} = 100 nA$')
# plt.legend()
plt.savefig('../plots/JJ1_JJ_3_MAR.png')

In [ ]:
I_range = 15e-6

I_to_fit_l = current[current < -I_range]
V_to_fit_l = voltage[current < -I_range]

I_to_fit_r = current[current > I_range]
V_to_fit_r = voltage[current > I_range]

fit_vals_l = np.polyfit(I_to_fit_l, V_to_fit_l, 1)
fit_vals_r = np.polyfit(I_to_fit_r, V_to_fit_r, 1)

R_N = (fit_vals_l[0] + fit_vals_r[0])/2

I_exs_l = -fit_vals_l[1] / fit_vals_l[0]
I_exs_r = -fit_vals_r[1] / fit_vals_r[0]

I_exs = (I_exs_r - I_exs_l)/2

In [ ]:
print(I_exs * R_N * 1e3)
print(I_exs * R_N * 1e3 / 0.49)

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), constrained_layout = True)
# ax.plot(1e3*voltage[0:-1], R_filt)
# ax.plot(1e3*voltage, R_filt)
ax.plot(1e6*current, 1e3*gaussian_filter(voltage, sigma=(1)))

ax.plot(1e6*current, 1e3*(current*fit_vals_l[0]+fit_vals_l[1]), linestyle = ':', color= 'black',linewidth = 0.9)

ax.axhline(0, linestyle = '-', color= 'black',linewidth = 0.7)

ax.set_xlabel(r'I ($\mu$A)', fontsize = 16)
ax.set_ylabel(r'V (mV)', fontsize = 16)

# plt.title('')

ax.set_ylim(-5, 5)
# ax.set_xlim(-15, 15)

# # inset axes....
# axins = ax.inset_axes([0.05, 0.65, 0.3, 0.3])
# axins.plot(1e6*current, 1e3*gaussian_filter(voltage, sigma=(1)))
# # sub region of the original image
# x1, x2, y1, y2 = -0.3, 0.3, -0.1, 0.1
# axins.set_xlim(x1, x2)
# axins.set_ylim(y1, y2)
# # axins.set_xticklabels('')
# axins.set_yticklabels('')
# plt.setp(axins.get_xticklabels(), fontsize=10)

# ax.indicate_inset_zoom(axins)

ax.annotate(r'$I_{exc}$='+'{:.1f}'.format(1e6*I_exs)+r'$\mu$A', xy=(1e6*I_exs_l, 0),  xycoords='data',
            xytext=(1e6*I_exs_l, -3), textcoords='data',
            arrowprops=dict(facecolor='black', width=0.1, headwidth = 6),
            horizontalalignment='right', verticalalignment='top',fontsize = 14
            )

ax.text(3, -2, r'$\frac{I_{exc}R_N}{\Delta} = 0.62 $', {'color': 'C0', 'fontsize': 16})
ax.text(3, -3, r'$Z = 0.71 $', {'color': 'C0', 'fontsize': 16})
plt.savefig('../plots/JJ1_JJ_3_MAR_1.png', transparency = True)

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), constrained_layout = True)
# ax.plot(1e3*voltage[0:-1], R_filt)
# ax.plot(1e3*voltage, R_filt)
ax.plot(1e3*voltage, gaussian_filter(R_AC, sigma=(1)), color = 'orange' )

# ax.axhline(0, linestyle = '-', color= 'black',linewidth = 0.7)

ax.set_ylabel(r'dV/dI ($\Omega$)', fontsize = 16)
ax.set_xlabel(r'V (mV)', fontsize = 16)

# plt.title('')



ax.annotate(r'$2\Delta$', xy=(1, 225),  xycoords='data',
            xytext=(1.5, 215), textcoords='data',
            arrowprops=dict(facecolor='black', width=0.1, headwidth = 6),
            horizontalalignment='right', verticalalignment='top',fontsize = 14
            )

ax.yaxis.set_ticks([270, 320, 370])

delta = 0.49
ax.axvline(2*delta)
ax.axvline(2*delta/2)
ax.axvline(2*delta/3)
# ax.axvline(2*delta/4)

ax.axvline(-2*delta)
ax.axvline(-2*delta/2)
ax.axvline(-2*delta/3)
# ax.axvline(-2*delta/4)

ax.set_ylim(200, 280)
ax.set_xlim(-2, 2)
plt.savefig('../plots/JJ1_JJ_3_MAR_2.png', transparency = True)

In [ ]:
fig, ax = plt.subplots()
# ax.plot(1e3*voltage[0:-1], R_filt)
# ax.plot(1e3*voltage, R_filt)
ax.plot(1e3*voltage, gaussian_filter(R_AC, sigma=(1)), label = 'lockin R')

ax.set_ylim(260, 380)
# ax.set_ylim(210, 250)
# ax.set_xlim(-2, 2)
ax.set_ylabel(r'$\frac{d \: \mathrm{V}}{d \: \mathrm{I}}$ ($\Omega$)', fontsize = 15)
ax.set_xlabel(r'Voltage (mV)', fontsize = 12)
plt.legend()
plt.title('BL3_JJ_2, $\mathrm{I}_\mathrm{AC} = 10 nA$')
plt.savefig('../plots/BL3_JJ_2_MAR.png')

In [ ]:
fig, ax = plt.subplots()

voltage = voltage[R_AC>10]
R_AC = R_AC[R_AC>10]

ax.plot(1e3*voltage, gaussian_filter(1e3/R_AC, sigma=(1)), label = 'lockin R')

# ax.set_ylim(210, 300)
ax.set_ylim(3.2, 3.8)
ax.set_xlim(-1.5, 1.5)
ax.set_ylabel(r'$\frac{d \: \mathrm{I}}{d \: \mathrm{V}}$ (mS)', fontsize = 15)
ax.set_xlabel(r'Voltage (mV)', fontsize = 12)
plt.title('BL3_JJ_2, $\mathrm{I}_\mathrm{AC} = 10 nA$')
plt.legend()
plt.savefig('../plots/BL3_JJ_2_MAR_2.png')

### MAR vs gate

In [ ]:
dataset = qc.load_by_id(101)
dataset.get_parameter_data().keys()

In [ ]:
dataset = qc.load_by_id(101)

JJ_id = 2

voltage = dataset.get_parameter_data()['meas_voltage_K{}'.format(JJ_id)]['meas_voltage_K{}'.format(JJ_id)]
current = dataset.get_parameter_data()['meas_voltage_K{}'.format(JJ_id)]['appl_current']
Vgate = dataset.get_parameter_data()['meas_voltage_K{}'.format(JJ_id)]['appl_TG']
V_lockin = dataset.get_parameter_data()['meas_voltage_R_Lockin{}'.format(JJ_id)]['meas_voltage_R_Lockin{}'.format(JJ_id)]

current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']


N_gate = np.size(np.unique(Vgate))
N_current = np.size(np.unique(current))

if np.size(voltage) < N_gate * N_current:
    N_gate = N_gate - 1
    voltage = voltage[0:N_gate * N_current]
    current = current[0:N_gate * N_current]
    Vgate = Vgate[0:N_gate * N_current]
    V_lockin = V_lockin[0:N_gate * N_current]
    

voltage = voltage.reshape(N_gate, N_current)
current = current.reshape(N_gate, N_current)
Vgate = Vgate.reshape(N_gate, N_current)
V_lockin = V_lockin.reshape(N_gate, N_current)

R_lockin = V_lockin / current_AC

In [ ]:
R_lockin_filt = gaussian_filter(R_lockin, sigma=(1))

In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap('hot')

pixel_width = Vgate[1,0]-Vgate[0,0]

im = ax.pcolormesh(Vgate-pixel_width/2, 1e3*voltage, 
                   R_lockin_filt, cmap = cmap, vmin=270,vmax=390)
fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)\
ax.set_ylim(-2, 2)
ax.set_ylabel(r'Voltage (mV)')
ax.set_xlabel(r'Gate (V)')
plt.title('BL3_JJ_{}'.format(JJ_id))
plt.savefig('../plots/BL3_JJ_{}_MAR_Gate_1.png'.format(JJ_id))
plt.show()

In [ ]:
fig, ax = plt.subplots()

Gate_id = 0
ax.plot(1e3*voltage[Gate_id,:], R_lockin_filt[Gate_id,:], label = 'Vgate = {:.2f}'.format(Vgate[Gate_id,0]))

Gate_id = 1
ax.plot(1e3*voltage[Gate_id,:], R_lockin_filt[Gate_id,:], label = 'Vgate = {:.2f}'.format(Vgate[Gate_id,0]))

Gate_id = 3
ax.plot(1e3*voltage[Gate_id,:], R_lockin_filt[Gate_id,:], label = 'Vgate = {:.2f}'.format(Vgate[Gate_id,0]))

ax.set_xlabel(r'Voltage (mV)')
ax.set_ylabel(r'Resistance $(\Omega)$')

ax.set_ylim(275, 315)
ax.set_xlim(-1.2, 1.2)
plt.legend()
plt.title('BL3_JJ_{}'.format(JJ_id))
plt.savefig('../plots/BL3_JJ_{}_MAR_Gate_3.png'.format(JJ_id))
plt.show()

In [ ]:
dataset = qc.load_by_id(101)

JJ_id = 1

voltage = dataset.get_parameter_data()['meas_voltage_K{}'.format(JJ_id)]['meas_voltage_K{}'.format(JJ_id)]
current = dataset.get_parameter_data()['meas_voltage_K{}'.format(JJ_id)]['appl_current']
Vgate = dataset.get_parameter_data()['meas_voltage_K{}'.format(JJ_id)]['appl_TG']
V_lockin = dataset.get_parameter_data()['meas_voltage_R_Lockin{}'.format(JJ_id)]['meas_voltage_R_Lockin{}'.format(JJ_id)]

current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']


N_gate = np.size(np.unique(Vgate))
N_current = np.size(np.unique(current))

if np.size(voltage) < N_gate * N_current:
    N_gate = N_gate - 1
    voltage = voltage[0:N_gate * N_current]
    current = current[0:N_gate * N_current]
    Vgate = Vgate[0:N_gate * N_current]
    V_lockin = V_lockin[0:N_gate * N_current]
    

voltage = voltage.reshape(N_gate, N_current)
current = current.reshape(N_gate, N_current)
Vgate = Vgate.reshape(N_gate, N_current)
V_lockin = V_lockin.reshape(N_gate, N_current)

R_lockin = V_lockin / current_AC

In [ ]:
R_lockin_filt = gaussian_filter(R_lockin, sigma=(1))

In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap('hot')

pixel_width = Vgate[1,0]-Vgate[0,0]

im = ax.pcolormesh(Vgate-pixel_width/2, 1e3*voltage, 
                   R_lockin_filt, cmap = cmap, vmin=210,vmax=320)
fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)\
ax.set_ylim(-2, 2)
ax.set_ylabel(r'Voltage (mV)')
ax.set_xlabel(r'Gate (V)')
plt.title('BL3_JJ_{}'.format(JJ_id))
plt.savefig('../plots/BL3_JJ_{}_MAR_Gate_1.png'.format(JJ_id))
plt.show()

In [ ]:
fig, ax = plt.subplots()

Gate_id = 0
ax.plot(1e3*voltage[Gate_id,:], R_lockin_filt[Gate_id,:], label = 'Vgate = {:.2f}'.format(Vgate[Gate_id,0]))

Gate_id = 1
ax.plot(1e3*voltage[Gate_id,:], R_lockin_filt[Gate_id,:], label = 'Vgate = {:.2f}'.format(Vgate[Gate_id,0]))

Gate_id = 3
ax.plot(1e3*voltage[Gate_id,:], R_lockin_filt[Gate_id,:], label = 'Vgate = {:.2f}'.format(Vgate[Gate_id,0]))

ax.set_xlabel(r'Voltage (mV)')
ax.set_ylabel(r'Resistance $(\Omega)$')

ax.set_ylim(210, 250)
ax.set_xlim(-1.2, 1.2)
plt.legend()
plt.title('BL3_JJ_{}'.format(JJ_id))
plt.savefig('../plots/BL3_JJ_{}_MAR_Gate_3.png'.format(JJ_id))
plt.show()

In [ ]:
import matplotlib.image as mpimg
from matplotlib import rc
import matplotlib.ticker as ticker
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator

In [ ]:
SMALL_SIZE = 6
MEDIUM_SIZE = 8
BIGGER_SIZE = 13

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=14)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

cmap = plt.get_cmap('hot')
levels = MaxNLocator(nbins=15).tick_values(0, 2e-6)
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

im = ax.pcolormesh(field*1e3, current*1e9, dif_voltage, cmap = cmap,  norm=norm)
fig.colorbar(im,ax=ax)
#plt.grid()
ax.set_ylabel(r'current (nA)')
ax.set_xlabel(r'field (mT)')
plt.tight_layout()
#plt.savefig('../plots/fraunhofer_TL3.png')
plt.show()

In [ ]:
from scipy.ndimage import gaussian_filter

In [ ]:
dif_voltage_filt = gaussian_filter( dif_voltage, sigma=(0.1,2))
dif_current_filt = gaussian_filter( dif_current, sigma=(0.1,2))

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4))

cmap = plt.get_cmap('hot')
levels = MaxNLocator(nbins=15).tick_values(0, 2e3)
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

im = ax.pcolormesh((field+0.3e-3)*1e3, current*1e9, dif_voltage_filt/dif_current_filt, cmap = cmap,  norm=norm)
cbar = fig.colorbar(im,ax=ax)
cbar.set_label('Resistance ($\Omega$)', rotation=90)
#plt.grid()
ax.set_ylabel(r'Current (nA)')
ax.set_xlabel(r'Magnetic field (mT)')
plt.tight_layout()


pi = 3.14159
offsetfield = 0.3e-3
fluxq = 2.068e-15
surfacearea = 2.25e-12
surfacearea2 = 2e-12
testcurrent = abs(np.sinc((field+offsetfield)*surfacearea/fluxq))
plt.plot((field+offsetfield)*1e3,70*testcurrent)
testcurrent2 = abs(np.sinc((field+offsetfield)*surfacearea2/fluxq))
plt.plot((field+offsetfield)*1e3,70*testcurrent2,'y')
plt.savefig('../plots/fraunhofer_TL3_filt5.pdf')
plt.show()


In [ ]:
pi = 3.14159
fluxq = 2.068e-15
surfacearea = 2e-12
testcurrent = abs(np.sinc(field*surfacearea/fluxq))
plt.plot(field,200*testcurrent)
plt.show()

In [ ]:
math.pi

In [ ]:
max = dif_voltage.max()

In [ ]:
max

dif_voltage.max()

In [ ]:
np.where(dif_voltage == max)

In [ ]:
field[41,0]*1000

In [ ]:
dataset = qc.load_by_id(23)
dataset.exp_name

In [ ]:
axes, cbaxes = plot_dataset(dataset)

In [ ]:
voltage_DC = dataset.get_parameter_data()['voltage_DC']['voltage_DC']
current_DC = dataset.get_parameter_data()['voltage_DC']['current']

In [ ]:
voltage_AC = dataset.get_parameter_data()['voltage_AC']['voltage_AC']

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(voltage_DC[:1999], np.diff(current_DC[0:2000]) / np.diff(voltage_DC[0:2000]) )
#ax.set_xlim(6,12)
ax.set_ylim(-0.5, 0.5)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(voltage_DC[:], gaussian_filter(voltage_AC[:], sigma=(2)) )
ax.set_xlim(-4e-3,-0.0e-3)
ax.set_ylim(4e-6, 6e-6)
delta = -1.3e-3
ax.axvline(delta)
ax.axvline(delta/2)
#ax.axvline(delta/3)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(current_DC, voltage_DC)
ax.set_xlim(-300e-9,300e-9)
ax.set_ylim(-0.15e-3, 0.2e-3)
plt.show()

In [ ]:
def fit_within_range(x,y,x_min,x_max):
    x_fit = x[(x>x_min)&(x<x_max)]
    y_fit = y[(x>x_min)&(x<x_max)]
    fit_values = np.polyfit(x_fit,y_fit,1)
    return fit_values

In [ ]:
fit_lim_left = [-3e-6, -2e-6]
fit_lim_right = [2e-6, 3e-6]

In [ ]:
a_l, b_l = fit_within_range(current_DC, voltage_DC, fit_lim_left[0], fit_lim_left[1])

In [ ]:
a_r, b_r = fit_within_range(current_DC, voltage_DC, fit_lim_right[0], fit_lim_right[1])

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(current_DC, voltage_DC)
ax.plot(current_DC, current_DC*a_l + b_l)
ax.plot(current_DC, current_DC*a_r + b_r)
ax.set_xlim(-400e-9,400e-9)
ax.set_ylim(-0.2e-3, 0.3e-3)
plt.show()

In [ ]:
excess_current = -(b_l/a_l - b_r/a_r)/2

In [ ]:
excess_current #in Amps

In [ ]:
a_l, b_l

In [ ]:
a_r, b_r

In [ ]:
dataset = qc.load_by_id(27)
voltage = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']
field = dataset.get_parameter_data()['meas_voltage_K1']['cryomag_A_field']

N_field = np.size(np.unique(field))
N_current = np.size(np.unique(current))

if np.size(voltage) < N_field * N_current:
    N_field = N_field - 1
    voltage = voltage[0:N_field * N_current]
    current = current[0:N_field * N_current]
    field = field[0:N_field * N_current]
    

voltage = voltage.reshape(N_field, N_current)
current = current.reshape(N_field, N_current)
field = field.reshape(N_field, N_current)

dif_voltage = np.diff(voltage, axis = 1)
dif_current = np.diff(current, axis = 1)
R = dif_voltage/dif_current
difdif_voltage = np.diff(dif_voltage, axis = 1)
R_filt = gaussian_filter( R, sigma=(0,1))

I_step = current[0,1] - current[0,0]
R_sm = smooth_dir(voltage, N_points=6) / I_step #scale the derivative

In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap('hot')

pixel_width = field[1,0]-field[0,0]
pixel_height = current[0,1]-current[0,0]

im = ax.pcolormesh(1e3*(field-pixel_width/2), 1e9 * (current-pixel_height/2), 
                   R_sm, cmap = cmap, vmin=0,vmax=500)
fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)\
# ax.set_xlim(-2, 2)
ax.set_ylabel(r'Current (nA)')
ax.set_xlabel(r'Field (mT)')
plt.title('JJ1_JJ_1')
# plt.savefig('../plots/R(I,B)_JJ1_JJ_1_b.png')
plt.show()

In [ ]:
import matplotlib.image as mpimg
from matplotlib import rc
import matplotlib.ticker as ticker
from matplotlib.colors import Normalize
import matplotlib.colors as colors

In [ ]:
import matplotlib

In [ ]:
SMALL_SIZE = 6
MEDIUM_SIZE = 8
BIGGER_SIZE = 10

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=10)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=10)    # fontsize of the tick labels
plt.rc('ytick', labelsize=10)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

class MidpointNormalize(colors.Normalize):
	"""
	Normalise the colorbar so that diverging bars work there way either side from a prescribed midpoint value)

	e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-100, vmax=100))
	"""
	def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
		self.midpoint = midpoint
		colors.Normalize.__init__(self, vmin, vmax, clip)

	def __call__(self, value, clip=None):
		# I'm ignoring masked values and all kinds of edge cases to make a
		# simple example...
		x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
		return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3))

cmap = plt.get_cmap('summer')

pixel_width = field[1,0]-field[0,0]
pixel_height = current[0,1]-current[0,0]

im = ax.pcolormesh(1e3*(field-pixel_width/2), 1e9 * (current-pixel_height/2), 
                   R_sm, cmap = cmap, vmin=0,vmax=400)
# fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)\
ax.set_xlim(-3, 3)
ax.set_ylabel(r'Current (nA)')
ax.set_xlabel(r'Field (mT)')

axins = ax.inset_axes([0.5, 1.05, 0.5, 0.05])

cbar = plt.colorbar(im,
cax=axins,
orientation='horizontal',
ticks = [0, 400],
use_gridspec=True
#drawedges=True
)
cbar.ax.xaxis.set_label_position('top')
cbar.ax.xaxis.tick_top()
axins.set_xlabel(r'dV/dI($\Omega$)', fontsize = 10)
# cbar.ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.3f'))
axins.tick_params(labelsize = 8)
# plt.title('JJ1_JJ_1')
cbar.outline.set_linewidth(1)
# ax.set_ylim(-2, 2)
# ax.set_xlim(-250, -75)

# start, end = ax.get_xlim()
# ax.xaxis.set_ticks(np.arange(start, end, 50))
# start, end = ax.get_ylim()
# ax.yaxis.set_ticks(np.arange(start, end+1, 1))

plt.tight_layout()
# plt.savefig('s5_6.pdf', format='pdf', transparent=True)
plt.savefig('../plots/R(I,B)_JJ1.png', format='png', transparent=True, dpi=300)

plt.show()

# Plots for report


## TLMs


In [ ]:
# Data for plotting

# M06-20-17.2 21
lengths21 = np.array([16, 33/3, 9, 6, 6])
r_s21 = np.array([3076, 7698/3, 1968, 1857, 2118])

# M06-20-17.2 31
lengths31 = np.array([12, 21/2, 9, 6])
r_s31 = np.array([566, 944/2, 417, 338])

#M07-10-19.1-3 41
lenghts41 = np.array([16,12,12,9,9,6,6,3])
r_s41 = np.array([642, 476, 479, 372, 364, 258, 261, 140])

#M06-18-19.1 33
lengths33 = np.array([28/2,12,9,9,6,6,3])
r_s33 = np.array([681/2, 299, 235, 233, 162, 159, 99])


np.savez('TLM_Rs_vs_lengths')

In [ ]:
lengths = lengths21
r_s = r_s21

fit_values = np.polyfit(lengths,r_s,1)
    
x_s = np.linspace(-1, 17, 100)
fig, ax = plt.subplots(1, figsize = (10,6))
ax.scatter(lengths, r_s,color = 'black',label='R from IV')
ax.plot(x_s, x_s*fit_values[0] + fit_values[1], label = 'fit')
ax.legend()
ax.set_ylabel('R ($\Omega$)')
ax.set_xlabel('2DEG length ($\mu m$)')
ax.set_title('M07-10-19.1-3 41')
ax.set_xlim(0, 17)
ax.set_ylim(0, 4000)
# plt.savefig('../plots/R_vs_length_fit_41.png')

## JJs

### load data

In [ ]:
# for samving images
save_dir = 'M:/tnw/ns/qt/2D Topo/code/personal_scripts/Mark/'
# load data
files = []
dirpath = 'M:\\tnw\\ns\\qt\\2D Topo\\samples\\InSbAs_epiAl\\M07-10-19.1_3\\11'
sample_name = dirpath.split('\\')[-1]
wafer_name = dirpath.split('\\')[-2]
db_path = dirpath + '\\data'

# r=root, d=directories, f = files
for r, d, f in os.walk(db_path):
    for file in f:
        if '.db' in file:
            files.append(os.path.join(r, file))

for f in files:
    print(f)

In [ ]:
db_path = os.path.join(db_path, 'M07-10-19.1_3_11_2021-04-19_01.db')
initialise_or_create_database_at(db_path)

In [ ]:
qc.experiments()

### IV + MAR

In [ ]:
dataset = qc.load_by_id(64)
voltage = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
voltage_AC = dataset.get_parameter_data()['meas_voltage_Lockin2']['meas_voltage_Lockin2']
current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']

# lockin = dataset.get_parameter_data()['meas_voltage_R_Lockin1']['meas_voltage_R_Lockin1']
current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']
# current_AC = 1e-9
R_AC = voltage_AC/current_AC
R_ACf = gaussian_filter(R_AC,sigma=1)

JJ = 'JJTRb' #to denote device and contacts
np.savez(save_dir + 'IV_+Iexc_({})_{}'.format(wafer_name + '_' + sample_name, JJ))


In [ ]:
voltage_AC

In [ ]:
# save_dir + 'IV_+Iexc_({})_{}.png'.format(wafer_name + '_' + sample_name, JJ)

In [ ]:
#IC
Imax = np.size(current)
for i in range(round(Imax/2),Imax):
    if R_AC[i] > 0.01:
        Ic = current[i]
        break
Voffset = voltage[round(Imax/2)]

voltage_cor = voltage - Voffset

# excess current
I_range = 20e-6

I_to_fit_l = current[current < -I_range]
V_to_fit_l = voltage_cor[current < -I_range]

I_to_fit_r = current[current > I_range]
V_to_fit_r = voltage_cor[current > I_range]

fit_vals_l = np.polyfit(I_to_fit_l, V_to_fit_l, 1)
fit_vals_r = np.polyfit(I_to_fit_r, V_to_fit_r, 1)

R_N = (fit_vals_l[0] + fit_vals_r[0])/2

I_exs_l = -fit_vals_l[1] / fit_vals_l[0]
I_exs_r = -fit_vals_r[1] / fit_vals_r[0]

I_exs = (I_exs_r - I_exs_l)/2

In [ ]:
R_AC[round(Imax/2)]

In [ ]:
fig, ax = plt.subplots(constrained_layout = False)
ax.plot(1e6*current, 1e3*voltage)

In [ ]:
fig, ax = plt.subplots(constrained_layout = False)
ax.plot(1e6*current, 1e3*voltage)
ax.plot(1e6*current, 1e3*(current*fit_vals_l[0]+fit_vals_l[1]), linestyle = ':', color= 'black',linewidth = 0.9)
ax.axhline(0, linestyle = '-', color= 'black',linewidth = 0.7)

ax.annotate(r'$I_{exc}$='+'{:.1f}'.format(1e6*I_exs)+r'$\mu$A', xy=(1e6*I_exs_l, 0),  xycoords='data',
            xytext=(1e6*I_exs_l, -1), textcoords='data',
            arrowprops=dict(facecolor='black', width=0.1, headwidth = 6),
            horizontalalignment='right', verticalalignment='top',fontsize = 14
            )

# ax.text(3, -2, r'$\frac{I_{exc}R_N}{\Delta} = 0.62 $', {'color': 'C0', 'fontsize': 16})
# ax.text(3, -3, r'$Z = 0.71 $', {'color': 'C0', 'fontsize': 16})

# ax.set_ylim(200, 280)
# ax.set_ylim(210, 250)
# ax.set_xlim(-2, 2)
# ax.set_ylabel(r'$\frac{d \: \mathrm{V}}{d \: \mathrm{I}}$ ($\Omega$)', fontsize = 15)
ax.set_ylabel('V (mV)')
ax.set_xlabel(r'current ($\mu A$)', fontsize = 12)
plt.title(wafer_name +  ' ' + sample_name + ' '+ JJ)
# plt.legend()
plt.savefig(save_dir + 'plots/' + 'IV_+Iexc_({})_{}.png'.format(wafer_name + '_' + sample_name, JJ))

fig, ax = plt.subplots(constrained_layout = False)
ax.plot(1e6*current, 1e3*voltage)
ax.plot(1e6*current, 1e3*voltage_cor)
ax.axhline(0, linestyle = '-', color= 'black',linewidth = 0.7)


ax.annotate(r'$I_{c}$='+'{:.1f}'.format(1e6*Ic)+r'$\mu$A', xy=(1e6*Ic, Voffset*1e3),  xycoords='data',
            xytext=(1e6*Ic, Voffset*1e3 - 0.2), textcoords='data',
            arrowprops=dict(facecolor='black', width=0.1, headwidth = 6),
            horizontalalignment='right', verticalalignment='top',fontsize = 14
            )

ax.set_ylim(-0.5, 0.5)
ax.set_xlim(-5, 5)
ax.set_ylabel('V (mV)')
ax.set_xlabel(r'current ($\mu A$)', fontsize = 12)
plt.title(wafer_name +  ' ' + sample_name + ' ' + JJ)
plt.savefig(save_dir + 'plots/' + 'IV_+Ic_({})_{}.png'.format(wafer_name + '_' + sample_name, JJ))

fig, ax = plt.subplots(constrained_layout = False)
ax.plot(1e6*current, R_ACf)
# ax.plot(1e3*voltage, R_ACf)
ax.axhline(0, linestyle = '-', color= 'black',linewidth = 0.7)
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)
# ax.set_ylim(0, 0.5)
# ax.set_xlim(-10, 10)
ax.set_ylabel(r'$\frac{dV}{dI}$ ($\Omega$)', fontsize = 12)
ax.set_xlabel(r'$I_{bias}$ ($\mu A$)', fontsize = 12)
# ax.set_xlabel(r'$V_{dc}$ ($mV$)', fontsize = 12)
# plt.title(wafer_name +  ' ' + sample_name+ ' ' + JJ)
# plt.savefig(save_dir + 'plots/' + 'dVdI_vsI_({})_{}_for_rapport.png'.format(wafer_name + '_' + sample_name, JJ),bbox_inches='tight')



fig, ax = plt.subplots(constrained_layout = False)
ax.plot(1e3*voltage, R_ACf)
ax.axhline(0, linestyle = '-', color= 'black',linewidth = 0.7)
ax.set_ylim(50,100)
ax.set_xlim(0, 1.5)
ax.set_ylabel('dV/dI ($\Omega$)')
ax.set_xlabel(r'Vdc ($mV$)', fontsize = 12)
# plt.title(wafer_name +  ' ' + sample_name + ' ' + JJ)
# plt.savefig(save_dir + 'plots/' + 'dVdI_+MAR_({})_{}_for_report.png'.format(wafer_name + '_' + sample_name, JJ),bbox_inches='tight')



### dV/dI vs B

In [ ]:
dataset = qc.load_by_id(29)
voltage = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
# voltage_AC = dataset.get_parameter_data()['meas_voltage_Lockin1']['meas_voltage_Lockin1']
current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']
field = dataset.get_parameter_data()['meas_voltage_K1']['cryomag_B_field']
#cryomag_field for janis, cryomag_B_field for vec janis, magnetic_field_yoko for yoko, z_field for fristi


N_field = np.size(np.unique(field))
N_current = np.size(np.unique(current))

if np.size(voltage) < N_field * N_current:
    N_field = N_field - 1
    voltage = voltage[0:N_field * N_current]
    current = current[0:N_field * N_current]
    field = field[0:N_field * N_current]
    

voltage = voltage.reshape(N_field, N_current)
current = current.reshape(N_field, N_current)
field = field.reshape(N_field, N_current)

dI = current[0,1] - current[0,0]
dR = smooth_dir(voltage,N_points = 2)/dI

# current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']
# current_AC = 1e-9
# R_AC = voltage_AC/current_AC

# np.savez('/plots/dVdI_vs_B_{}_{}'.format(wafer_name + sample_name, JJ))



In [ ]:
JJ = 'JJa'

In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap('hot')

pixel_width = field[1,0]-field[0,0]
pixel_height = current[0,1]-current[0,0]

im = ax.pcolormesh(1e3*field, 1e9 * current, 
                   dR, cmap = cmap, shading = 'nearest'
                   ,vmin=0,vmax=300
                  )
cbar = fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)\
# ax.set_xlim(-2, 2)
ax.set_ylabel(r'$I_{bias}$ (nA)')
ax.set_xlabel(r'B (mT)')
cbar.set_label(r'$\frac{dV}{dI}$ ($\Omega$)', rotation=90)

# plt.title(wafer_name +  ' ' + sample_name + ' ' + JJ)
plt.savefig(save_dir + '/plots/dVdI_vs_B_({})_{}.png'.format(wafer_name + '_' + sample_name, JJ),bbox_inches='tight')
plt.show()

### dV/dI vs TG

In [ ]:
dataset = qc.load_by_id(72)
voltage = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
# voltage_AC = dataset.get_parameter_data()['meas_voltage_Lockin1']['meas_voltage_Lockin1']
current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']
TG = dataset.get_parameter_data()['meas_voltage_K1']['appl_TG']

xas = TG
yas = current

N_xas = np.size(np.unique(xas))
N_yas = np.size(np.unique(yas))

if np.size(voltage) < N_xas * N_yas:
    N_field = N_xas - 1
    voltage = voltage[0:N_xas * N_yas]
    current = current[0:N_xas * N_yas]
    field = field[0:N_xas * N_yas]
    

voltage = voltage.reshape(N_xas, N_yas)
xas = xas.reshape(N_xas, N_yas)
yas = yas.reshape(N_xas, N_yas)

TG = xas
current = yas

dI = current[0,1] - current[0,0]
dR = smooth_dir(voltage,N_points = 2)/dI

# current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']
# current_AC = 1e-9
# R_AC = voltage_AC/current_AC


np.savez('/plots/dVdI_vs_TG_{}_{}'.format(wafer_name + sample_name, JJ))


In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap('hot')
im = ax.pcolormesh(TG, 1e9 * current, dR, cmap = cmap, shading = 'nearest'
                   , vmin=0,vmax=300
                  )
cbar = fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)
# ax.set_xlim(-10, 10)
ax.set_ylabel(r'Current (nA)')
ax.set_xlabel(r'V gate (V)')
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90)
plt.title(wafer_name +  ' ' + sample_name + ' ' + JJ)
plt.savefig(save_dir + '/plots/dVdI_vs_TG_({})_{}.png'.format(wafer_name + '_' + sample_name, JJ))
plt.show()

## HB

In [ ]:
# for samving images
save_dir = 'M:/tnw/ns/qt/2D Topo/code/personal_scripts/Mark/'
# load data
files = []
dirpath = 'M:\\tnw\\ns\\qt\\2D Topo\\samples\\InSbAs_epiAl\\M08-22-19.1_3\\22'
sample_name = dirpath.split('\\')[-1]
wafer_name = dirpath.split('\\')[-2]
db_path = dirpath + '\\data'

# r=root, d=directories, f = files
for r, d, f in os.walk(db_path):
    for file in f:
        if '.db' in file:
            files.append(os.path.join(r, file))

for f in files:
    print(f)

In [ ]:
db_path = os.path.join(db_path, 'M08-22-19.1_3_22_2021-08-26_01.db')
initialise_or_create_database_at(db_path)

In [ ]:
qc.experiments()

### Mobility density

In [ ]:
electron_charge = 1.60217662e-19
R_K = 25812.8

In [ ]:
dataset = qc.load_by_id(40)
# voltage = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
voltage1_AC = dataset.get_parameter_data()['meas_voltage_Lockin1']['meas_voltage_Lockin1']
voltage2_AC = dataset.get_parameter_data()['meas_voltage_Lockin2']['meas_voltage_Lockin2']
field = dataset.get_parameter_data()['meas_voltage_Lockin1']['z_field']
#cryomag_field for janis, magnetic_field_yoko for yoko, z_field for fristi

TG = dataset.get_parameter_data()['meas_voltage_Lockin1']['appl_TG']

current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']
xas = field # slow parameter
yas = TG #sweep parameter

N_xas = np.size(np.unique(xas))
N_yas = np.size(np.unique(yas))

# if np.size(voltage) < N_xas * N_yas:
#     N_field = N_xas - 1
#     voltage = voltage[0:N_xas * N_yas]
#     current = current[0:N_xas * N_yas]
#     field = field[0:N_xas * N_yas]
    

voltage1_AC = voltage1_AC.reshape(N_xas, N_yas)
voltage2_AC = voltage2_AC.reshape(N_xas, N_yas)
xas = xas.reshape(N_xas, N_yas)
yas = yas.reshape(N_xas, N_yas)

N_field = N_xas
N_gate = N_yas
field = xas
v_gate = yas


Rxy = voltage1_AC/current_AC
Rxx = -voltage2_AC/current_AC

# np.savez(save_dir + 'Fan_diagram_{}_{}'.format(wafer_name + sample_name, HB))

In [ ]:
Rxx = voltage1_AC/current_AC
Rxy = -voltage2_AC/current_AC

In [ ]:
def fit_within_range(x,y,x_min,x_max):
    x_fit = x[(x>x_min)&(x<x_max)]
    y_fit = y[(x>x_min)&(x<x_max)]
    fit_values = np.polyfit(x_fit,y_fit,1)
    return fit_values

In [ ]:
gate_vals = v_gate[1,:]
n_vals = np.zeros_like(gate_vals)

for gate_idx in np.arange(0,N_gate):
    a, b = fit_within_range(field[:,gate_idx], Rxy[:,gate_idx], -0.5, 0.5)
    n_vals[gate_idx] = 1 / (electron_charge * a)
    
gate_vals = gate_vals[3:]
n_vals = n_vals[3:]

In [ ]:
from scipy.interpolate import interp1d

# density_func = interp1d(gate_vals, n_vals, kind='linear', fill_value="extrapolate")
density_func = np.poly1d(np.polyfit(gate_vals, n_vals, 3))
density_inverse_function = np.poly1d(np.polyfit(n_vals, gate_vals, 3))
density_func2 = np.poly1d(np.polyfit(gate_vals, n_vals*1e-4, 3))
density_inverse_function2 = np.poly1d(np.polyfit(n_vals*1e-4, gate_vals, 3))

In [ ]:
plt.rc('axes', titlesize=14)
plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=14)
plt.rc('figure', titlesize=14)

fig, ax = plt.subplots(figsize=(8,6))
ax.scatter(gate_vals[::3], n_vals[::3]*1e-4, label = 'exp data')
gates_new = np.linspace(-2, 0.6, 40)
ax.plot(gates_new, density_func(gates_new)*1e-4, color = 'purple', label = 'qubic fit')
# densities_new = 1e15 * np.linspace(1, 3.2, 40)
# ax.plot(density_inverse_function(densities_new), densities_new, color = 'orange', label = 'inverse qubic fit')
ax.legend()
ax.set_xlabel(r'Gate (V)')
ax.set_ylabel(r'Density $(\mathrm{cm}^{-2})$')
plt.title('BR HB density vs gate from AC Hall, exp {}\n{}'.format(dataset.run_id, dataset.path_to_db.split('\\')[-1]))
# plt.savefig('../plots/BR_density_gate_AC_Hall.png')
plt.show()

### Fan diagram

In [ ]:
HB = 'HB2'

In [ ]:
dataset = qc.load_by_id(198)
dataset.snapshot['station']['parameters']

In [ ]:
dataset = qc.load_by_id(58)
# voltage = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
voltage1_AC = dataset.get_parameter_data()['meas_voltage_Lockin1']['meas_voltage_Lockin1']
voltage2_AC = dataset.get_parameter_data()['meas_voltage_Lockin2']['meas_voltage_Lockin2']
field = dataset.get_parameter_data()['meas_voltage_Lockin1']['z_field']
#cryomag_field for janis, magnetic_field_yoko for yoko, z_field for fristi

TG = dataset.get_parameter_data()['meas_voltage_Lockin1']['appl_TG']

current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']
xas = field # slow parameter
yas = TG #sweep parameter

N_xas = np.size(np.unique(xas))
N_yas = np.size(np.unique(yas))

if np.size(voltage1_AC) < N_xas * N_yas:
    N_field = N_xas - 1
    voltage1_AC = voltage1_AC[0:N_xas * N_yas]
    voltage2_AC = voltage2_AC[0:N_xas * N_yas]
#     current = current[0:N_xas * N_yas]
    TG = TG[0:N_xas * N_yas]
    field = field[0:N_xas * N_yas]
    

voltage1_AC = voltage1_AC.reshape(N_xas, N_yas)
voltage2_AC = voltage2_AC.reshape(N_xas, N_yas)
xas = xas.reshape(N_xas, N_yas)
yas = yas.reshape(N_xas, N_yas)

field = xas
TG = yas


R1_AC = voltage1_AC/current_AC
R2_AC = voltage2_AC/current_AC

# np.savez('M:/tnw/ns/qt/2D Topo/code/personal_scripts/Mark/Fan_diagram_{}_HB2'.format(wafer_name + sample_name), R1_AC = Rxx, R2_AC = Rxy, density_func2 = density_func2, density_inverse_function2 = density_inverse_function2, TG = TG, field = field )

In [ ]:
# # #load npz file instead of database
# HB = 'HB2'
# wafer_n = 'M07-10-19.1_3'
# sample_n = '23'
# npzfile = np.load(save_dir + 'Fan_diagram_{}_{}.npz'.format(wafer_n + sample_n, HB))
# wafer_name = wafer_n
# sample_name = sample_n

# R1_AC = npzfile['R1_AC']
# R2_AC = npzfile['R2_AC']
# TG = npzfile['TG']
# field = npzfile['field']
# density_func2 = np.poly1d(npzfile['density_func2'])
# density_inverse_fuction2= np.poly1d(npzfile['density_inverse_function2'])

In [ ]:
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20) 

In [ ]:
HB = 'HB1'

In [ ]:
fig, ax = plt.subplots(figsize = [12, 6])

cmap = plt.get_cmap('hot')
im = ax.pcolormesh(TG, field, gaussian_filter(R2_AC,sigma = 1), cmap = cmap, shading = 'nearest'
                   , vmin=0,vmax=6000
                  )
cbar = fig.colorbar(im,ax=ax)

# ax.set_ylim(-190, 190)
# ax.set_xlim(-10, 10)
ax.set_xlabel(r'gate (V)',fontsize = 20)
ax.set_ylabel(r'field (T)', fontsize = 20)
secax = ax.secondary_xaxis('top', functions=(density_func2,density_inverse_function2))
secax.set_xlabel(r'elctron density $(cm^{-2})$',fontsize =20)
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90, fontsize = 20)
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
bbox_inches='tight'
plt.savefig(save_dir + 'plots/' + 'Fan_diagram_collect1_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = [8, 4])

cmap = plt.get_cmap('turbo')
im = ax.pcolormesh(TG, field, R_K/R1_AC, cmap = cmap, shading = 'nearest'
                   , vmin=0,vmax=10
                  )
cbar = fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)
# ax.set_xlim(-10, 10)
ax.set_xlabel(r'gate (V)')
ax.set_ylabel(r'field (T)')
secax = ax.secondary_xaxis('top', functions=(density_func2,density_inverse_function2))

# secax.set_xlabel(r'elctron density $(cm^{-2})$')

cbar.set_label(r'dV/dI $(\frac{h}{e^2})$', rotation=90)
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
bbox_inches='tight'
# plt.savefig(save_dir + 'plots/' + 'Fan_diagram_WB_Rxy_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()

fig, ax = plt.subplots(figsize = [8, 4])

cmap = plt.get_cmap('hot')
im = ax.pcolormesh(TG, field, gaussian_filter(R2_AC,sigma = 1), cmap = cmap, shading = 'nearest'
                   , vmin=0,vmax=6000
                  )
cbar = fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)
# ax.set_xlim(-10, 10)
ax.set_xlabel(r'gate (V)')
ax.set_ylabel(r'field (T)')
secax = ax.secondary_xaxis('top', functions=(density_func2,density_inverse_function2))
secax.set_xlabel(r'elctron density $(cm^{-2})$')
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90)
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
bbox_inches='tight'
# plt.savefig(save_dir + 'plots/' + 'Fan_diagram_WB_Rxx_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()

In [ ]:
HB2

In [ ]:
3e-6*R_K/2

In [ ]:
# Linecut
B = 7
i = np.argmin(abs(field[:,1] - B))
fig, ax = plt.subplots(figsize = [6,3],constrained_layout = True)
ax.plot(TG[i,:],R_K/R1_AC[i,:])
ax.axhline(1, linestyle = '-', color= 'black',linewidth = 0.7)
ax.axhline(2, linestyle = '-', color= 'black',linewidth = 0.7)
ax.axhline(3, linestyle = '-', color= 'black',linewidth = 0.7)
ax.axhline(4, linestyle = '-', color= 'black',linewidth = 0.7)


fig, ax = plt.subplots(figsize = [6,3],constrained_layout = True)
R2_AC_F = gaussian_filter(R2_AC,sigma = 1.4)
ax.plot(TG[i,:],-R2_AC_F[i,:],linewidth = 2,color = 'black')
ax.set_xlabel('$V_{gate}(V)$')
ax.set_ylabel('$dV/dI (\Omega)$')
plt.savefig(save_dir + 'plots/' + 'Rxx_fan_cut_({})_{}.png'.format(wafer_name + '_' + sample_name, HB))



### Andreev Enhancement

In [ ]:
# AE vs Gate
dataset = qc.load_by_id(15)
voltage = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
voltage_AC = dataset.get_parameter_data()['meas_voltage_Lockin1']['meas_voltage_Lockin1']
current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']
TG = dataset.get_parameter_data()['meas_voltage_K1']['appl_TG']

xas = TG
yas = current

N_xas = np.size(np.unique(xas))
N_yas = np.size(np.unique(yas))

if np.size(voltage) < N_xas * N_yas:
    N_field = N_xas - 1
    voltage = voltage[0:N_xas * N_yas]
    current = current[0:N_xas * N_yas]
    field = field[0:N_xas * N_yas]
    

voltage = voltage.reshape(N_xas, N_yas)
voltage_AC = voltage_AC.reshape(N_xas,N_yas)
xas = xas.reshape(N_xas, N_yas)
yas = yas.reshape(N_xas, N_yas)

TG = xas
current = yas

dI = current[0,1] - current[0,0]
dR = smooth_dir(voltage,N_points = 2)/dI

current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']
# current_AC = 1e-9

R_AC = gaussian_filter(voltage_AC/current_AC, sigma=(3.5))
G_AC = 1/R_AC

# np.savez(save_dir + 'dVdI_vs_TG_{}_{}'.format(wafer_name + sample_name, HB))


In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap('inferno')
im = ax.pcolormesh( voltage, TG, R_AC, cmap = cmap, shading = 'nearest'
#                    , vmin=0,vmax=300
                  )
cbar = fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)
# ax.set_xlim(-10, 10)
# ax.set_xlabel(r'Current ($\mu A$)')
ax.set_xlabel(r'Vdc ($V$)')
ax.set_ylabel(r'V gate (V)')
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90)
plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
# plt.savefig(save_dir + '/plots/dVdI_vs_TG_({})_{}.png'.format(wafer_name + '_' + sample_name, HB))
plt.show()

In [ ]:
np.argmin(TG)

In [ ]:
fig, ax = plt.subplots(constrained_layout = False)
n,n2 = np.shape(R_AC)
colors = plt.cm.copper(np.linspace(0,1,n))
for i in range(1,n):
    ax.plot(1e6*np.transpose(current[i,:]), np.transpose(R_AC[i,:]),color = colors[i,:])

tg = 0
fig, ax = plt.subplots(constrained_layout = False)
i0 = np.argmin(abs(TG[:,1 - tg]))

ax.plot(1e6*np.transpose(current[i0,:]), np.transpose(R_AC[i0,:]),color = 'black')
ax.set_title('TG = 0')

fig, ax = plt.subplots(constrained_layout = False)
ax.plot(1e6*np.transpose(current[i0,:]), 1000*np.transpose(G_AC[i0,:]),color = 'black')
ax.set_title('TG = 0')
ax.set_ylabel('G (mS)')
ax.set_xlim(-5,5)
ax.set_ylim(3.25,4)

Gmax =np.max(G_AC[i0,:]*1e3)
Gn = 3.58
Genh= (Gmax-Gn)/Gn*100
ax.axhline(Gn, linestyle = '--', color= 'black',linewidth = 0.7)
ax.axhline(Gmax, linestyle = '--', color= 'black',linewidth = 0.7)
ax.text(1.5,3.8, r'$ Genh = {:.2f}\% $'.format(Genh), {'color': 'black', 'fontsize': 16})




In [ ]:
np.shape(colors)

In [ ]:
############################################################################################################################

In [ ]:
############################################################################################################################

In [ ]:
dataset = qc.load_by_id(15)
voltage = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
voltage_AC = dataset.get_parameter_data()['meas_voltage_Lockin1']['meas_voltage_Lockin1']
current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']

# lockin = dataset.get_parameter_data()['meas_voltage_R_Lockin1']['meas_voltage_R_Lockin1']
current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']
# current_AC = 1e-9
R_AC = voltage_AC/current_AC
R_cor = gaussian_filter(R_AC,sigma = 0.6) - 267
Vdc = gaussian_filter(voltage - 267*current, sigma = 0.6)
G = 1/R_cor
Gn = G[np.size(current)-1]
# np.savez(save_dir + 'Andreev_enhancement({})_{}.p¿ng'.format(wafer_name + '_' + sample_name, HB))

In [ ]:
fig, ax = plt.subplots(constrained_layout = False,figsize = [8,4])
ax.plot(current*1e6, R_AC, linewidth = 2, color = 'blue')
ax.plot(current*1e6, R_cor, linewidth = 2, color = 'black')

# ax.plot(1e6*current, 1e3*(current*fit_vals_l[0]+fit_vals_l[1]), linestyle = ':', color= 'black',linewidth = 0.9)
# ax.axhline(0, linestyle = '-', color= 'black',linewidth = 0.7)


# ax.set_ylim(210, 250)
# ax.set_xlim(-2, 2)
ax.set_ylabel(r'$dV/dI$ $(\Omega)$', fontsize = 20)
ax.set_xlabel(r'current ($\mu A$)', fontsize = 20)
# plt.title(wafer_name +  ' ' + sample_name + ' '+ HB)
# plt.legend()
plt.savefig(save_dir + '/plots/dVdI_AE_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')

fig, ax = plt.subplots(constrained_layout = False,figsize = [8,4])
ax.plot(Vdc*1e3+0.1, G*1e3, linewidth = 2, color = 'black')
# ax.plot(1e6*current, 1e3*(current*fit_vals_l[0]+fit_vals_l[1]), linestyle = ':', color= 'black',linewidth = 0.9)
# ax.axvline(0.135, linestyle = '-', color= 'black',linewidth = 1)
# ax.axvline(-0.135, linestyle = '-', color= 'black',linewidth = 1)


# ax.set_ylim(210, 250)
# ax.set_xlim(-2, 2)
ax.set_ylabel(r'G $(mS)$', fontsize = 20)
ax.set_xlabel(r'$V^*_{DC}$ $(mV)$', fontsize = 20)
# plt.title(wafer_name +  ' ' + sample_name + ' '+ HB)
# plt.legend()
plt.savefig(save_dir + '/plots/dIdV_AE_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')


### Single Lockin Rxx/Rxy

In [ ]:
dataID = 7
dataset = qc.load_by_id(dataID)
voltage1 = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
voltage1_AC = dataset.get_parameter_data()['meas_voltage_Lockin1']['meas_voltage_Lockin1']
voltage2 = dataset.get_parameter_data()['meas_voltage_K2']['meas_voltage_K2']
voltage2_AC = dataset.get_parameter_data()['meas_voltage_Lockin2']['meas_voltage_Lockin2']
TG = dataset.get_parameter_data()['meas_voltage_K1']['appl_TG_dac10']

bias = current_AC = dataset.snapshot['station']['parameters']['appl_current']['value']
current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']
# current_AC = 1e-9
R_AC1 = voltage1_AC/current_AC
R_AC1f = gaussian_filter(R_AC1,sigma=2)
R_AC2 = voltage2_AC/current_AC
R_AC2f = gaussian_filter(R_AC2,sigma=2)

HB = 'TR' #to denote device and contacts
# np.savez(save_dir + 'IV_+AC_Rxx_Rxy_({})_{}'.format(wafer_name + '_' + sample_name, JJ))


In [ ]:
# voltage1_AC

In [ ]:
fig, ax = plt.subplots(constrained_layout = False)
ax.plot(TG, R_AC1)

plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)

ax.set_ylabel(r'R')
ax.set_xlabel(r'V gate (V)')

fig, ax = plt.subplots(constrained_layout = False)
ax.plot(TG, R_K/R_AC2)
ax.axhline(1, linestyle = '-', color= 'black',linewidth = 0.7)
ax.axhline(2, linestyle = '-', color= 'black',linewidth = 0.7)
ax.axhline(3, linestyle = '-', color= 'black',linewidth = 0.7)
ax.axhline(4, linestyle = '-', color= 'black',linewidth = 0.7)

ax.set_ylabel(r'R')
ax.set_xlabel(r'V gate (V)')

### SG vs TG

In [ ]:
dataID = 138
dataset = qc.load_by_id(dataID)
voltage1 = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
voltage2 = dataset.get_parameter_data()['meas_voltage_K2']['meas_voltage_K2']
voltage1_AC = dataset.get_parameter_data()['meas_voltage_Lockin1']['meas_voltage_Lockin1']
voltage2_AC = dataset.get_parameter_data()['meas_voltage_Lockin2']['meas_voltage_Lockin2']

SG1 = dataset.get_parameter_data()['meas_voltage_K1']['split_gate_R']
TG = dataset.get_parameter_data()['meas_voltage_K1']['top_gate']
current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']

#anti clip
Nv = np.size(voltage1)
for i in range(0,Nv):
    if voltage1[i] >= 10e-3:
        voltage1[i] = np.sign(i - Nv)*10e-3
    if voltage2[i] >= 10e-3:
        voltage2[i] = np.sign(i - Nv)*10e-3


voltage1 = np.array(voltage1,dtype=np.float)
voltage2 = np.array(voltage2,dtype=np.float)


yas = TG
xas = SG1

N_xas = np.size(np.unique(xas))
N_yas = np.size(np.unique(yas))

if np.size(voltage1) < N_xas * N_yas:
    N_field = N_xas - 1
    voltage1 = voltage1[0:N_xas * N_yas]
    voltage2 = voltage2[0:N_xas * N_yas]    
    current = current[0:N_xas * N_yas]
    field = field[0:N_xas * N_yas]
    

voltage1_AC = voltage1_AC.reshape(N_xas, N_yas)
voltage2_AC = voltage2_AC.reshape(N_xas, N_yas)
voltage1 = voltage1.reshape(N_xas, N_yas)
voltage2 = voltage2.reshape(N_xas, N_yas)
xas = xas.reshape(N_xas, N_yas)
yas = yas.reshape(N_xas, N_yas)

TG = yas
SG1 = xas

dR1_AC = voltage1_AC/current_AC
dR2_AC = voltage2_AC/current_AC


# np.savez('/plots/dVdI_vs_TG_{}_{}'.format(wafer_name + sample_name, JJ))


In [ ]:
dR1_AC_f = -gaussian_filter(dR1_AC,0.8)
dR2_AC_f = gaussian_filter(dR2_AC,2)


In [ ]:
nu = 4
fig, ax = plt.subplots(figsize = [5, 4])
plt.tight_layout()
cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(TG, SG, dR1_AC_f, cmap = cmap, shading = 'nearest'
                   , vmin=-600,vmax=600
                  )
cbar = fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)
# ax.set_xlim(-10, 10)
ax.set_xlabel(r'Top Gate (V)',fontsize = 20)
ax.set_ylabel(r'Split Gate $(V)$',fontsize = 20)
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90, fontsize = 20)
# secax = ax.secondary_xaxis('top', functions=(timesXY,inv_timesXY))
# secax.set_xlabel(r'I $\times$ Rxy $(mV)$')

plt.savefig(save_dir + '/plots/SG_TG_lockin({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()

fig, ax = plt.subplots(figsize = [8,6])
plt.tight_layout()
cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(TG, SG, dR2_AC, cmap = cmap, shading = 'nearest'
                   , vmin=-50,vmax=50
                  )
cbar = fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)
# ax.set_xlim(-10, 10)
# secax = ax.secondary_xaxis('top', functions=(timesXY,inv_timesXY))
# secax.set_xlabel(r'I $\times$ Rxy $(mV)$')
ax.set_xlabel(r'Top Gate (V)',fontsize = 20)
ax.set_ylabel(r'Split Gate $(V)$',fontsize = 20)
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90, fontsize = 20)

# cbar.set_label(r'dV/dI ($h/e^{2}$)', rotation=90)
plt.title('$V_2$')
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
# plt.savefig(save_dir + '/plots/dVdI_vs_TG_Oscillations_Lockin_LeftSN_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')

In [ ]:
# bias cut
tg = -0.975
i = np.argmin(abs(TG[1,:] - tg))
fig, ax = plt.subplots(figsize = [8,4],constrained_layout = False)
ax.plot(SG1[:,i], dR1_AC_f[:,i],color = 'black')
ax.plot(SG[:,i], dR1[:,i],color = 'blue')
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)

ax.set_ylabel(r'dV/dI $(\Omega)$',fontsize = 22)
ax.set_xlabel(r'Split Gate $(V)$',fontsize = 22)
# secax = ax.secondary_xaxis('top', functions=(timesXY,inv_timesXY))
# secax.set_xlabel(r'I $\times$ Rxy $(mV)$',fontsize = 22)
# ax.legend(['TG= {:.4f}'.format(TG[i,1])])
ax.legend(['Lockin','dV/dI from DC'])
# plt.title('$V_1$ ')
plt.savefig(save_dir + '/plots/SG_Lockin_vs_dc({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')



### Lockin vs TG

In [ ]:
dataID = 176
dataset = qc.load_by_id(dataID)
voltage1 = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
voltage2 = dataset.get_parameter_data()['meas_voltage_K2']['meas_voltage_K2']
voltage1_AC = dataset.get_parameter_data()['meas_voltage_Lockin1']['meas_voltage_Lockin1']
voltage2_AC = dataset.get_parameter_data()['meas_voltage_Lockin2']['meas_voltage_Lockin2']

current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']
TG = dataset.get_parameter_data()['meas_voltage_K1']['appl_TG_dac10']
current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']

#anti clip
Nv = np.size(voltage1)
for i in range(0,Nv):
    if voltage1[i] >= 10e-3:
        voltage1[i] = np.sign(i - Nv)*10e-3
    if voltage2[i] >= 10e-3:
        voltage2[i] = np.sign(i - Nv)*10e-3


voltage1 = np.array(voltage1,dtype=np.float)
voltage2 = np.array(voltage2,dtype=np.float)


xas = TG
yas = current

N_xas = np.size(np.unique(xas))
N_yas = np.size(np.unique(yas))

if np.size(voltage1) < N_xas * N_yas:
    N_field = N_xas - 1
    voltage1 = voltage1[0:N_xas * N_yas]
    voltage2 = voltage2[0:N_xas * N_yas]    
    current = current[0:N_xas * N_yas]
    field = field[0:N_xas * N_yas]
    

voltage1_AC = voltage1_AC.reshape(N_xas, N_yas)
voltage2_AC = voltage2_AC.reshape(N_xas, N_yas)
voltage1 = voltage1.reshape(N_xas, N_yas)
voltage2 = voltage2.reshape(N_xas, N_yas)
xas = xas.reshape(N_xas, N_yas)
yas = yas.reshape(N_xas, N_yas)

TG = xas
current = yas

dR1_AC = voltage1_AC/current_AC
dR2_AC = voltage2_AC/current_AC
dR1_AC_f = gaussian_filter(dR1_AC,0.5)


# np.savez('/plots/dVdI_vs_TG_{}_{}'.format(wafer_name + sample_name, JJ))


In [ ]:
np.shape(dR1_AC)
HB = 'HB2'

In [ ]:
dR1_AC_f = gaussian_filter(dR1_AC,0.5)
dR2_AC_f = gaussian_filter(dR2_AC,0.5)

In [ ]:
def timesXY(I):
    V = 1e3*(I*1e-6)*R_K/nu
    return V
def inv_timesXY(V):
    I = (V/1e3)*nu/R_K*1e6
    return I
nu = 2

In [ ]:
nu = 4
fig, ax = plt.subplots(figsize = [8, 6])
plt.tight_layout()
cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(1e9 * current, TG, dR1_AC_f, cmap = cmap, shading = 'nearest'
                   , vmin=-100,vmax=100
                  )
cbar = fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)
# ax.set_xlim(-10, 10)
ax.set_xlabel(r'Current (nA)')
ax.set_ylabel(r'V gate (V)')
secax = ax.secondary_xaxis('top', functions=(timesXY,inv_timesXY))
secax.set_xlabel(r'I $\times$ Rxy $(mV)$')
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90)
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
plt.title('$V_1$')

# plt.savefig(save_dir + '/plots/dVdI_vs_TG_Oscillations_Lockin_BottomSN({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()

fig, ax = plt.subplots(figsize = [8,6])
plt.tight_layout()
cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(1e9 * current, TG, dR2_AC_f, cmap = cmap, shading = 'nearest'
                   , vmin=-50,vmax=50
                  )
cbar = fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)
# ax.set_xlim(-10, 10)
secax = ax.secondary_xaxis('top', functions=(timesXY,inv_timesXY))
secax.set_xlabel(r'I $\times$ Rxy $(mV)$')
ax.set_xlabel(r'Current (nA)')
ax.set_ylabel(r'V gate (V)')
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90)

# cbar.set_label(r'dV/dI ($h/e^{2}$)', rotation=90)
plt.title('$V_2$')
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
# plt.savefig(save_dir + '/plots/dVdI_vs_TG_Oscillations_Lockin_LeftSN_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')

In [ ]:
# bias cut
tg = -1.135
i = np.argmin(abs(TG[:,1] - tg))
i= i
fig, ax = plt.subplots(figsize = [6,3],constrained_layout = False)
ax.plot(1e9*current[i,:], dR1_AC_f[i,:])

# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)

ax.set_ylabel(r'dV/dI $(\Omega)$')
ax.set_xlabel(r'current $(nA)$')
secax = ax.secondary_xaxis('top', functions=(timesXY,inv_timesXY))
secax.set_xlabel(r'I $\times$ Rxy $(mV)$')
ax.legend(['TG= {:.3f}'.format(TG[i,1])])
plt.title('$V_1$ ')

plt.savefig(save_dir + '/plots/Linecut_V1_Lockin_vs_bias_Oscillations_LeftSN_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')

fig, ax = plt.subplots(figsize = [6,3],constrained_layout = False)
ax.plot(1e9*current[i,:], dR2_AC_f[i,:])

# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)

ax.set_ylabel(r'dV/dI $(\Omega)$')
ax.set_xlabel(r'current $(nA)$')
secax = ax.secondary_xaxis('top', functions=(timesXY,inv_timesXY))
secax.set_xlabel(r'I $\times$ Rxy $(mV)$')
plt.title('$V_2$')
ax.legend(['TG= {:.3f}'.format(TG[i,1])])
plt.savefig(save_dir + '/plots/Linecut_V2_Lockin_vs_bias_Oscillations_LeftSN_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')






# fig, ax = plt.subplots(figsize = [6,3],constrained_layout = False)
# ax.plot(1e9*current[i,:], voltage1[i,:]*1e3)

# # plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)

# ax.set_ylabel(r'Vdc (mV)')
# ax.set_xlabel(r'V gate (V)')
# secax = ax.secondary_xaxis('top', functions=(timesXY,inv_timesXY))
# secax.set_xlabel(r'I $\times$ Rxy $(mV)$')

In [ ]:
TG[1,2]


### Lockin Field Gate sweep

In [ ]:
dataID = 217
dataset = qc.load_by_id(dataID)
# voltage1 = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
# voltage2 = dataset.get_parameter_data()['meas_voltage_K2']['meas_voltage_K2']
voltage1_AC = dataset.get_parameter_data()['meas_voltage_Lockin1']['meas_voltage_Lockin1']
voltage2_AC = dataset.get_parameter_data()['meas_voltage_Lockin2']['meas_voltage_Lockin2']

field = dataset.get_parameter_data()['meas_voltage_Lockin1']['z_field']
TG = dataset.get_parameter_data()['meas_voltage_Lockin1']['appl_TG_dac10']
current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']


# voltage1 = np.array(voltage1,dtype=np.float)
# voltage2 = np.array(voltage2,dtype=np.float)


xas = field
yas = TG

N_xas = np.size(np.unique(xas))
N_yas = np.size(np.unique(yas))

# if np.size(voltage1) < N_xas * N_yas:
#     N_field = N_xas - 1
# #     voltage1 = voltage1[0:N_xas * N_yas]
# #     voltage2 = voltage2[0:N_xas * N_yas]    
# #     current = current[0:N_xas * N_yas]
#     TG = TG[0:N_xas * N_yas]
#     field = field[0:N_xas * N_yas]
    

voltage1_AC = voltage1_AC.reshape(N_xas, N_yas)
voltage2_AC = voltage2_AC.reshape(N_xas, N_yas)
# voltage1 = voltage1.reshape(N_xas, N_yas)
# voltage2 = voltage2.reshape(N_xas, N_yas)
xas = xas.reshape(N_xas, N_yas)
yas = yas.reshape(N_xas, N_yas)

TG = yas
field = xas

dR1_AC = voltage1_AC/current_AC
dR2_AC = voltage2_AC/current_AC
dR1_AC_f = gaussian_filter(dR1_AC,0.7)
dR2_AC_f = gaussian_filter(dR2_AC,0.7)


# np.savez('/plots/dVdI_vs_TG_{}_{}'.format(wafer_name + sample_name, JJ))


In [ ]:
# nu = 4
fig, axs = plt.subplots(1,2,figsize = [12, 6])
# plt.tight_layout()
ax = axs[0]
cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(TG,field,dR1_AC_f, cmap = cmap, shading = 'nearest'
                   , vmin=-30,vmax=30
                  )
# ax.set_ylim(-190, 190)
# ax.set_xlim(-10, 10)
ax.set_ylabel(r'B (T)')
ax.set_xlabel(r'TG (V) ')

cbar.set_label(r'dV/dI ($\Omega$)', rotation=90)
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
ax.set_title('$V_1$')

ax = axs[1]
cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(TG,field,dR2_AC_f, cmap = cmap, shading = 'nearest'
                   , vmin=-30,vmax=30
                  )
# ax.set_ylim(-190, 190)
# ax.set_xlim(-10, 10)
ax.set_xlabel(r'TG (V) ')
# ax.set_ylabel(r'B (T)')
cbar = fig.colorbar(im,ax=ax)
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90)
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
ax.set_title('$V_2$')


plt.savefig(save_dir + '/plots/dVdI_vs_B,TG_Oscillations_Lockin({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()


### Bias vs Field

In [ ]:
dataID = 96
dataset = qc.load_by_id(dataID)
voltage1 = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
voltage2 = dataset.get_parameter_data()['meas_voltage_K2']['meas_voltage_K2']
# voltage1_AC = dataset.get_parameter_data()['meas_voltage_Lockin1']['meas_voltage_Lockin1']
# voltage2_AC = dataset.get_parameter_data()['meas_voltage_Lockin2']['meas_voltage_Lockin2']

field = dataset.get_parameter_data()['meas_voltage_K1']['z_field']
current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']

TG = dataset.snapshot['station']['parameters']['top_gate']['value']
# current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']


# voltage1 = np.array(voltage1,dtype=np.float)
# voltage2 = np.array(voltage2,dtype=np.float)


xas = field
yas = current

N_xas = np.size(np.unique(xas))
N_yas = np.size(np.unique(yas))

# if np.size(voltage1) < N_xas * N_yas:
#     N_field = N_xas - 1
# #     voltage1 = voltage1[0:N_xas * N_yas]
# #     voltage2 = voltage2[0:N_xas * N_yas]    
# #     current = current[0:N_xas * N_yas]
#     TG = TG[0:N_xas * N_yas]
#     field = field[0:N_xas * N_yas]
    

# voltage1_AC = voltage1_AC.reshape(N_xas, N_yas)
# voltage2_AC = voltage2_AC.reshape(N_xas, N_yas)
voltage1 = voltage1.reshape(N_xas, N_yas)
voltage2 = voltage2.reshape(N_xas, N_yas)
xas = xas.reshape(N_xas, N_yas)
yas = yas.reshape(N_xas, N_yas)

current = yas
field = xas


dI = current[0,1] - current[0,0]
dR1 = smooth_dir(voltage1,N_points = 6)/dI
dR2 = smooth_dir(voltage2,N_points = 6)/dI

# dR1_AC = voltage1_AC/current_AC
# dR2_AC = voltage2_AC/current_AC
# dR1_AC_f = gaussian_filter(dR1_AC,0.7)
# dR2_AC_f = gaussian_filter(dR2_AC,0.7)


# np.savez('/plots/dVdI_vs_TG_{}_{}'.format(wafer_name + sample_name, JJ))


In [ ]:
# fig, ax = plt.subplots()
# plt.tight_layout()
# cmap = plt.get_cmap('hot')
# im = ax.pcolormesh(field, 1e9 * current, dR1, cmap = cmap, shading = 'nearest'
#                    , vmin=0,vmax=5
#                   )
# cbar = fig.colorbar(im,ax=ax)
# # ax.set_ylim(-190, 190)
# # ax.set_xlim(-10, 10)
# ax.set_ylabel(r'Current (nA)')
# ax.set_xlabel(r'V gate (V)')
# cbar.set_label(r'dV/dI ($\Omega$)', rotation=90)
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
# # plt.savefig(save_dir + '/plots/dVdI_vs_TG_({})_{}.png'.format(wafer_name + '_' + sample_name, JJ))
# plt.show()

fig, ax = plt.subplots()
plt.tight_layout()
cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(1e9 * current, field, -dR2, cmap = cmap, shading = 'nearest'
                   , vmin=-40,vmax=40
                  )
cbar = fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)
# ax.set_xlim(-10, 10)
ax.set_xlabel(r'Current (nA)')
ax.set_ylabel(r'B $(T)$')
# cbar.set_label(r'dV/dI ($h/e^{2}$)', rotation=90)
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90)
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
plt.savefig(save_dir + '/plots/dVdI_vs_I,B_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()

### dV/dI vs TG

In [ ]:
dataID = 78
dataset = qc.load_by_id(dataID)
voltage1 = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
voltage2 = dataset.get_parameter_data()['meas_voltage_K2']['meas_voltage_K2']
# voltage_AC = dataset.get_parameter_data()['meas_voltage_Lockin1']['meas_voltage_Lockin1']
current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']
TG = dataset.get_parameter_data()['meas_voltage_K1']['appl_TG_dac10']
# TG = dataset.get_parameter_data()['meas_voltage_K1']['delta_top_gate']
# TGS =  dataset.snapshot['station']['parameters']['top_gate']['value']


#anti clip
Nv = np.size(voltage1)
for i in range(0,Nv):
    if voltage1[i] >= 10e-3:
        voltage1[i] = np.sign(i - Nv)*10e-3
    if voltage2[i] >= 10e-3:
        voltage2[i] = np.sign(i - Nv)*10e-3


voltage1 = np.array(voltage1,dtype=np.float)
voltage2 = np.array(voltage2,dtype=np.float)


xas = TG
yas = current

N_xas = np.size(np.unique(xas))
N_yas = np.size(np.unique(yas))

if np.size(voltage1) < N_xas * N_yas:
    N_field = N_xas - 1
    voltage1 = voltage1[0:N_xas * N_yas]
    voltage2 = voltage2[0:N_xas * N_yas]    
    current = current[0:N_xas * N_yas]
    field = field[0:N_xas * N_yas]
    

voltage1 = voltage1.reshape(N_xas, N_yas)
voltage2 = voltage2.reshape(N_xas, N_yas)
xas = xas.reshape(N_xas, N_yas)
yas = yas.reshape(N_xas, N_yas)

TG = xas
current = yas

# dI = current[0,1] - current[0,0]
# dR1 = smooth_dir(voltage1,N_points = 8)/dI
# dR2 = smooth_dir(voltage2,N_points = 8)/dI


# current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']
# current_AC = 1e-9
# R_AC = voltage_AC/current_AC


# np.savez('/plots/dVdI_vs_TG_{}_{}'.format(wafer_name + sample_name, JJ))


In [ ]:
dI = current[0,1] - current[0,0]
dR1 = smooth_dir(voltage1,N_points = 12)/dI
dR2 = smooth_dir(voltage2,N_points = 12)/dI

In [ ]:
HB = 'HB2'

In [ ]:
dR1_f = gaussian_filter(dR1,sigma = 0.5)


In [ ]:
np.shape(voltage1)
np.shape(np.ones([81,301]))

In [ ]:
np.shape(np.matmul(np.reshape(voltage1[:,0],[81,1]),np.ones([1,81])))
# np.reshape(voltage1[:,0],[81,1])
# np.shape(np.ones([0,81]))

In [ ]:
voltage1_cor = voltage1 - np.matmul(np.reshape(voltage1[:,0],[81,1]),np.ones([1,301]))
voltage1_cor_fil = gaussian_filter(voltage1_cor,sigma = 0.5) 

In [ ]:
fig, ax = plt.subplots(figsize = [7, 4])
# fig, ax = plt.subplots()
plt.tight_layout()
cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(1e9 * current,TG,voltage1_cor_fil/(current+1e-9), cmap = cmap, shading = 'nearest'
                   , vmin=-10,vmax=10
                  )
cbar = fig.colorbar(im,ax=ax)
matplotlib.rc('xtick', labelsize=18) 
matplotlib.rc('ytick', labelsize=18) 

# ax.set_ylim(-190, 190)
# ax.set_xlim(0, 500)
# x1 = [771.4, -1.5999683]
# x2 = [490.8, -1.6009090]
# slope:
# ax.plot([x1[0], x2[0]],[x1[1],x2[1]], linewidth = 2, color = 'black')
# slope= (x1[1] - x2[1])/((x1[0] - x2[0])*1e-9)

ax.set_xlabel(r'Current (nA)',fontsize = 20)
ax.set_ylabel(r'$V_{gate}$ $(V)$',fontsize = 20)
# plt.title('slope = {:.2f} (kV/A)'.format(slope * 1e-3),fontsize = 20)
cbar.set_label(r'R ($\Omega$)', rotation=90, fontsize = 20)

# cbar.set_label(r'dV/dI ($h/e^{2}$)', rotation=90)
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
# plt.savefig(save_dir + '/plots/R_single_osc({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize = [7, 4])
# fig, ax = plt.subplots()
plt.tight_layout()
cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(1e9 * current,TG,voltage1_cor_fil*1e6, cmap = cmap, shading = 'nearest'
                   , vmin=-2,vmax=2
                  )
cbar = fig.colorbar(im,ax=ax)
matplotlib.rc('xtick', labelsize=18) 
matplotlib.rc('ytick', labelsize=18) 

# ax.set_ylim(-190, 190)
# ax.set_xlim(0, 500)
# x1 = [771.4, -1.5999683]
# x2 = [490.8, -1.6009090]
# slope:
# ax.plot([x1[0], x2[0]],[x1[1],x2[1]], linewidth = 2, color = 'black')
# slope= (x1[1] - x2[1])/((x1[0] - x2[0])*1e-9)

ax.set_xlabel(r'Current (nA)',fontsize = 20)
ax.set_ylabel(r'$V_{gate}$ $(V)$',fontsize = 20)
# plt.title('slope = {:.2f} (kV/A)'.format(slope * 1e-3),fontsize = 20)
cbar.set_label(r'V ($\mu V$)', rotation=90, fontsize = 20)

# cbar.set_label(r'dV/dI ($h/e^{2}$)', rotation=90)
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
# plt.savefig(save_dir + '/plots/V_single_osc({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()


In [ ]:
# bias cut
tg = -1.55
i = np.argmin(abs(TG[:,1] - tg))
fig, ax = plt.subplots(figsize = [8,4],constrained_layout = False)
ax.plot(1e9*current[i,:], voltage1_cor_fil[i,:]*1e6,color = 'black')

# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)

ax.set_ylabel(r'V $(\mu V)$',fontsize = 22)
ax.set_xlabel(r'current $(nA)$',fontsize = 22)
secax = ax.secondary_xaxis('top', functions=(timesXY,inv_timesXY))
secax.set_xlabel(r'I $\times$ Rxy $(mV)$',fontsize = 22)
ax.legend(['TG= {:.4f}'.format(TG[i,1])])
# plt.title('$V_1$ ')
# plt.savefig(save_dir + '/plots/Osc1_linecut({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')



In [ ]:
fig, ax = plt.subplots(figsize = [7, 4])
# fig, ax = plt.subplots()
plt.tight_layout()
cmap = plt.get_cmap('RdPu')
im = ax.pcolormesh(1e9 * current,TG,gaussian_filter(dR2,sigma=0.3), cmap = cmap, shading = 'nearest'
                   , vmin=0,vmax=2000
                  )
cbar = fig.colorbar(im,ax=ax)
matplotlib.rc('xtick', labelsize=18) 
matplotlib.rc('ytick', labelsize=18) 

ax.set_ylim(-1.1, -0.3)
# ax.set_xlim(0, 500)
# x1 = [771.4, -1.5999683]
# x2 = [490.8, -1.6009090]
# slope:
# ax.plot([x1[0], x2[0]],[x1[1],x2[1]], linewidth = 2, color = 'black')
# slope= (x1[1] - x2[1])/((x1[0] - x2[0])*1e-9)

ax.set_xlabel(r'Current (nA)',fontsize = 20)
ax.set_ylabel(r'$V_{gate}$ $(V)$',fontsize = 20)
# plt.title('slope = {:.2f} (kV/A)'.format(slope * 1e-3),fontsize = 20)
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90, fontsize = 20)

# cbar.set_label(r'dV/dI ($h/e^{2}$)', rotation=90)
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
plt.savefig(save_dir + '/plots/dVdI_LS_Large_B=5({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize = [7, 4])
# fig, ax = plt.subplots()
plt.tight_layout()
cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(1e9 * current, TGS + TG,dR1, cmap = cmap, shading = 'nearest'
                   , vmin=-20,vmax=20
                  )
cbar = fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)
# ax.set_xlim(0, 500)
# x1 = [489.6, -1.5999992]
# x2 = [437.6, -1.60084928]
# slope:
# ax.plot([x1[0], x2[0]],[x1[1],x2[1]], linewidth = 2, color = 'black')
# slope= (x1[1] - x2[1])/((x1[0] - x2[0])*1e-9)

ax.set_xlabel(r'Current (nA)',fontsize = 20)
ax.set_ylabel(r'$V_{gate}$ $(V)$',fontsize = 20)
plt.title('slope = {:.2f} (kV/A)'.format(slope * 1e-3),fontsize = 20)
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90, fontsize = 20)

# cbar.set_label(r'dV/dI ($h/e^{2}$)', rotation=90)
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
# plt.savefig(save_dir + '/plots/dV1dI_conf_G2_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()


In [ ]:
def timesXY(I):
    V = 1e3*(I*1e-9)*R_K/nu
    return V
def inv_timesXY(V):
    I = (V/1e3)*nu/R_K*1e9
    return I
nu = 2

In [ ]:
TGS+TG[:,i]

In [ ]:
i = np.argmin(abs(TGS+TG[:,i] - tg))
i

In [ ]:
v2_f = -gaussian_filter(voltage2[i,:],sigma = 1)
voff = v2_f[round(N_yas/2)]

In [ ]:
# bias cut
tg = -1.0614
i = np.argmin(abs(TGS+TG[:,1] - tg))
fig, ax = plt.subplots(figsize = [8,4],constrained_layout = False)
ax.plot(1e9*current[i,:], v2_f - voff,color = 'black')

# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)

ax.set_ylabel(r'dV/dI $(\Omega)$',fontsize = 22)
ax.set_xlabel(r'current $(nA)$',fontsize = 22)
secax = ax.secondary_xaxis('top', functions=(timesXY,inv_timesXY))
secax.set_xlabel(r'I $\times$ Rxy $(mV)$',fontsize = 22)
ax.legend(['TG= {:.4f}'.format(TGS + TG[i,1])])
# plt.title('$V_1$ ')
# plt.savefig(save_dir + '/plots/Osc1_linecut({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')



In [ ]:



fig, ax = plt.subplots() # figsize = [4, 6]
plt.tight_layout()
cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(1e9 * current,TG,  gaussian_filter(dR1,sigma=0.6), cmap = cmap, shading = 'nearest'
                   , vmin=-40,vmax=40
                  )
cbar = fig.colorbar(im,ax=ax)
# x1 = [514, -1.598]
# x2 = [385, -1.599966]

# # slope:
# ax.plot([x1[0], x2[0]],[x1[1],x2[1]], linewidth = 2, color = 'black')
# slope= (x1[1] - x2[1])/((x1[0] - x2[0])*1e-9)



# ax.set_ylim(-2, -1)
# ax.set_xlim(0, 500)
ax.set_xlabel(r'Current (nA)')
ax.set_ylabel(r'$V_{gate}$ $(V)$')
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90)
# plt.title('slope = {:.2f} (kV/A)'.format(slope * 1e-3))
# plt.title('B = 5.5T')
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
# plt.savefig(save_dir + '/plots/dV1dI_sym_v_=ASYM2_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()




## single dR from IV 

In [ ]:
def timesXY(I):
    V = 1e3*(I*1e-6)*R_K/nu
    return V
def inv_timesXY(V):
    I = (V/1e3)*nu/R_K*1e6
    return I
nu = 2

In [ ]:
3e-6*R_K/2

In [ ]:
dataID = 345
dataset = qc.load_by_id(dataID)
voltage1 = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
# voltage1_AC = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_']
# voltage2 = dataset.get_parameter_data()['meas_voltage_K2']['meas_voltage_K2']
current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']

dI = current[1] - current[0]
dR1 = smooth_dir_1D(voltage1,N_points = 36)/dI
# dR2 = smooth_dir_1D(voltage2,N_points = 8)/dI

fig, ax = plt.subplots(figsize = [12,6],constrained_layout = False)
# ax.plot(1e9*current, 1e3*voltage1)
ax.set_xlabel('Current $\mu A$')
ax.set_ylabel('dV/dI $(\Omega)$')
secax = ax.secondary_xaxis('top', functions=(timesXY,inv_timesXY))
secax.set_xlabel(r'I $\times$ Rxy $(mV)$')
ax.plot(1e6*current, dR1, color= 'black',linewidth = 1.5)

plt.savefig(save_dir + '/plots/dVdI_large_range_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')


## dV/dI vs temperature (M06-18-19.1 33)

In [ ]:
dataID = 383
dataset = qc.load_by_id(dataID)
voltage1 = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
voltage2 = dataset.get_parameter_data()['meas_voltage_K2']['meas_voltage_K2']
# voltage_AC = dataset.get_parameter_data()['meas_voltage_Lockin1']['meas_voltage_Lockin1']
current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']
# TG = dataset.get_parameter_data()['meas_voltage_K1']['appl_TG_dac10']
temp = dataset.get_parameter_data()['meas_voltage_K1']['lakeshore_C_temperature']
TG =  dataset.snapshot['station']['parameters']['appl_TG']['value']


# #anti clip
# Nv = np.size(voltage1)
# for i in range(0,Nv):
#     if voltage1[i] >= 10e-3:
#         voltage1[i] = np.sign(i - Nv)*10e-3
#     if voltage2[i] >= 10e-3:
#         voltage2[i] = np.sign(i - Nv)*10e-3


# voltage1 = np.array(voltage1,dtype=np.float)
# voltage2 = np.array(voltage2,dtype=np.float)


xas = temp
yas = current

N_xas = np.size(np.unique(xas))
N_yas = np.size(np.unique(yas))

if np.size(voltage1) < N_xas * N_yas:
    N_field = N_xas - 1
    voltage1 = voltage1[0:N_xas * N_yas]
    voltage2 = voltage2[0:N_xas * N_yas]    
    current = current[0:N_xas * N_yas]
    field = field[0:N_xas * N_yas]
    

voltage1 = voltage1.reshape(N_xas, N_yas)
voltage2 = voltage2.reshape(N_xas, N_yas)
xas = xas.reshape(N_xas, N_yas)
yas = yas.reshape(N_xas, N_yas)

temp = xas
current = yas



# current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']
# current_AC = 1e-9
# R_AC = voltage_AC/current_AC


# np.savez('/plots/dVdI_vs_TG_{}_{}'.format(wafer_name + sample_name, JJ))


In [ ]:
dI = current[0,1] - current[0,0]
dR1 = smooth_dir(voltage1,N_points = 48)/dI
dR2 = smooth_dir(voltage2,N_points = 48)/dI


In [ ]:
fig, ax = plt.subplots()
plt.tight_layout()
cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(1e9 * current,temp,  gaussian_filter(dR1,sigma = 0.5), cmap = cmap, shading = 'nearest'
                   , vmin=-10,vmax=10
                  )
cbar = fig.colorbar(im,ax=ax)
# x1 = [514, -1.598]
# x2 = [385, -1.599966]
# # slope:
# ax.plot([x1[0], x2[0]],[x1[1],x2[1]], linewidth = 2, color = 'black')
# slope= (x1[1] - x2[1])/((x1[0] - x2[0])*1e-9)
ax.set_ylim(0.260, 1)
# ax.set_xlim(-10, 10)
ax.set_xlabel(r'Current (nA)')
ax.set_ylabel(r'$V_{SG}$ $(V)$')
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90)
# plt.title('slope = {:.2f} (kV/A)'.format(slope * 1e-3))
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
# plt.savefig(save_dir + '/plots/dV1dI_vs_Split_Gates_regular_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()


fig, ax = plt.subplots()
plt.tight_layout()
cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(1e9 * current,temp,  gaussian_filter(dR2,sigma = 0.5), cmap = cmap, shading = 'nearest'
#                    , vmin=-20,vmax=20
                  )
cbar = fig.colorbar(im,ax=ax)
# x1 = [514, -1.598]
# x2 = [385, -1.599966]
# # slope:
# ax.plot([x1[0], x2[0]],[x1[1],x2[1]], linewidth = 2, color = 'black')
# slope= (x1[1] - x2[1])/((x1[0] - x2[0])*1e-9)
# ax.set_ylim(-190, 190)
# ax.set_xlim(-10, 10)
ax.set_xlabel(r'Current (nA)')
ax.set_ylabel(r'$V_{SG}$ $(V)$')
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90)
# plt.title('slope = {:.2f} (kV/A)'.format(slope * 1e-3))
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
# plt.savefig(save_dir + '/plots/dV1dI_vs_Split_Gates_regular_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()


In [ ]:
curi = np.argmin(abs(current[1,:]*1e9 + 48))
curi
Rpeak = np.zeros(N_xas)
valip = np.zeros(N_xas)
rng = 100
for i in range(0,N_xas):
    valip[i] = curi - rng + np.argmax((dR1[i,curi-rng:curi+rng]))
    
for i in range(0,N_xas):
#     Rpeak[i] = dR1[i,xp[i]]
    Rpeak[i] = np.max((dR1[i,curi-rng:curi+rng]))
#     xp[i] = np.argmax((dR1[i,curi-20:curi+20]))
    

fig, ax = plt.subplots(figsize = [12,6],constrained_layout = False)

ax.scatter(np.transpose(temp[:,curi])*1e9,Rpeak,color = 'black')
ax.set_xlabel('Temperature $(K)$',fontsize = 24)
ax.set_ylabel('$R_{peak}$')
ax.set

In [ ]:
curi = np.argmin(abs(current[1,:]*1e9 - 23))
curi
Rval = np.zeros(N_xas)
valiv = np.zeros(N_xas)
rng = 100
for i in range(0,N_xas):
    valiv[i] = curi - rng + np.argmin((dR1[i,curi-rng:curi+rng]))
    
for i in range(0,N_xas):
#     Rpeak[i] = dR1[i,xp[i]]
    Rval[i] = np.min((dR1[i,curi-rng:curi+rng]))
#     xp[i] = np.argmax((dR1[i,curi-20:curi+20]))
    

fig, ax = plt.subplots(figsize = [12,6],constrained_layout = False)

ax.scatter(np.transpose(temp[:,curi])*1e9,Rval,color = 'black')
ax.set_xlabel('Temperature $(K)$',fontsize = 24)
ax.set_ylabel('$R_{peak}$')
ax.set

In [ ]:
fig, ax = plt.subplots(figsize = [12,6],constrained_layout = False)

ax.scatter(np.transpose(temp[:,curi])*1e9,Rval/np.min(Rval),color = 'black')
ax.scatter(np.transpose(temp[:,curi])*1e9,Rpeak/np.max(Rpeak),color = 'blue')
ax.set_xlabel('Temperature $(K)$',fontsize = 20)
ax.legend(['Positive peak','Negative peak'],fontsize = 20)
ax.set_ylabel('$R_{peak}/R_{max_peak}$',fontsize = 20)

plt.savefig(save_dir + '/plots/Temp_dependence2_peaks_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')



In [ ]:
curi = np.argmin(abs(current[1,:]*1e9 - 48))
curi

In [ ]:
np.shape(current)

In [ ]:
vali = np.zeros(N_xas)
rng = 20
for i in range(0,N_xas):
    vali[i] = curi - rng + np.argmax((dR1[i,curi-rng:curi+rng]))

In [ ]:
np.shape(vali)
# np.shape(temp[:,1])

In [ ]:
fig, ax = plt.subplots(figsize=(4, 8))
plt.tight_layout()
cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(1e9 * current,temp,  gaussian_filter(dR1,sigma = 0.5), cmap = cmap, shading = 'nearest'
                   , vmin=-10,vmax=10
                  )
for ii in range(0,N_xas):
    iiiv = int(valiv[ii])
    iiip= int(valip[ii])
    ax.scatter(current[1,iiiv]*1e9,temp[ii,1], color = 'black', s = 20, marker = 'x')
    ax.scatter(current[1,iiip]*1e9,temp[ii,1], color = 'black', s = 20,  marker = 'x')

    
cbar = fig.colorbar(im,ax=ax)
# x1 = [514, -1.598]
# x2 = [385, -1.599966]
# # slope:
# ax.plot([x1[0], x2[0]],[x1[1],x2[1]], linewidth = 2, color = 'black')
# slope= (x1[1] - x2[1])/((x1[0] - x2[0])*1e-9)
ax.set_ylim(0.260, 1)
# ax.set_xlim(-10, 10)
ax.set_xlabel(r'Current (nA)',fontsize = 20)
ax.set_ylabel(r'$V_{SG}$ $(V)$',fontsize = 20)
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90,fontsize = 20)
# plt.title('slope = {:.2f} (kV/A)'.format(slope * 1e-3))
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
plt.savefig(save_dir + '/plots/Temp_dependence1_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()



## Temp depencence (M07-10-19.1_3 23)

In [ ]:
dataID = 106
dataset = qc.load_by_id(dataID)
voltage1 = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
voltage2 = dataset.get_parameter_data()['meas_voltage_K2']['meas_voltage_K2']
# voltage_AC = dataset.get_parameter_data()['meas_voltage_Lockin1']['meas_voltage_Lockin1']
current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']
# TG = dataset.get_parameter_data()['meas_voltage_K1']['appl_TG_dac10']
time = dataset.get_parameter_data()['meas_voltage_K1']['time']
# TG =  dataset.snapshot['station']['parameters']['appl_TG']['value']


# #anti clip
# Nv = np.size(voltage1)
# for i in range(0,Nv):
#     if voltage1[i] >= 10e-3:
#         voltage1[i] = np.sign(i - Nv)*10e-3
#     if voltage2[i] >= 10e-3:
#         voltage2[i] = np.sign(i - Nv)*10e-3


# voltage1 = np.array(voltage1,dtype=np.float)
# voltage2 = np.array(voltage2,dtype=np.float)


xas = time
yas = current

N_xas = np.size(np.unique(xas))
N_yas = np.size(np.unique(yas))

if np.size(voltage1) < N_xas * N_yas:
    N_field = N_xas - 1
    voltage1 = voltage1[0:N_xas * N_yas]
    voltage2 = voltage2[0:N_xas * N_yas]    
    current = current[0:N_xas * N_yas]
    field = field[0:N_xas * N_yas]
    

voltage1 = voltage1.reshape(N_xas, N_yas)
voltage2 = voltage2.reshape(N_xas, N_yas)
xas = xas.reshape(N_xas, N_yas)
yas = yas.reshape(N_xas, N_yas)

time = xas
current = yas



# current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']
# current_AC = 1e-9
# R_AC = voltage_AC/current_AC


# np.savez('/plots/dVdI_vs_TG_{}_{}'.format(wafer_name + sample_name, JJ))


In [ ]:
dI = current[0,1] - current[0,0]
dR1 = smooth_dir(voltage1,N_points = 6)/dI
dR2 = smooth_dir(voltage2,N_points = 6)/dI


In [ ]:
temp = np.load(save_dir + '/temp.npy')

In [ ]:

fig, ax = plt.subplots(figsize = (16,4))
plt.tight_layout()
cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(1e9 * current,temp, dR2, cmap = cmap, shading = 'nearest'
                   , vmin=-40,vmax=40
                  )
cbar = fig.colorbar(im,ax=ax)
# x1 = [514, -1.598]
# # slope:
# ax.plot([x1[0], x2[0]],[x1[1],x2[1]], linewidth = 2, color = 'black')
# slope= (x1[1] - x2[1])/((x1[0] - x2[0])*1e-9)
ax.set_ylim(24, 100)
ax.set_xlim(-550, 809)
ax.set_xlabel(r'Current (nA)',fontsize = 20)
ax.set_ylabel(r'Temperature $(mK)$',fontsize = 20)
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90,fontsize = 20)
# plt.title('slope = {:.2f} (kV/A)'.format(slope * 1e-3))
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
plt.savefig(save_dir + '/plots/Temp_dependence_Ivan_zoom_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()


In [ ]:
npi = [-537, -381, 295, 357, 585, 730]
ppi = [-515, -358, -227, 330, 631, 777]

In [ ]:
# np.shape(temp)
current[1,curi]
dR2[24,curi]

In [ ]:
c = 0
Rval = np.zeros([N_xas,np.size(npi)])
valiv = np.zeros([N_xas,np.size(npi)])
for ii in npi:
    curi = np.argmin(abs(current[1,:]*1e9 - ii))
#     print(curi)
    rng = 5
    for i in range(0,N_xas):
        valiv[i,c] = curi - rng + np.argmin((dR2[i,curi-rng:curi+rng]))

    for i in range(0,N_xas):
    #     Rpeak[i] = dR1[i,xp[i]]
        Rval[i,c] = np.min((dR2[i,curi-rng:curi+rng]))
    #     xp[i] = np.argmax((dR1[i,curi-20:curi+20]))
    c = c + 1


fig, ax = plt.subplots(figsize = [12,6],constrained_layout = False)
# ax.scatter(np.transpose(temp[:,curi]),Rval[:,3])
for i in range(0,c-1):
    ax.scatter(np.transpose(temp[:,curi]),Rval[:,i])
ax.set_xlabel('Temperature $(K)$',fontsize = 24)
ax.set_ylabel('$R_{peak}$')


In [ ]:
c = 0
Rpeak = np.zeros([N_xas,np.size(npi)])
valip = np.zeros([N_xas,np.size(npi)])
for ii in ppi:
    curi = np.argmin(abs(current[1,:]*1e9 - ii))
#     print(curi)
    rng = 20
    for i in range(0,N_xas):
        valip[i,c] = curi - rng + np.argmax((dR2[i,curi-rng:curi+rng]))

    for i in range(0,N_xas):
    #     Rpeak[i] = dR1[i,xp[i]]
        Rpeak[i,c] = np.max((dR2[i,curi-rng:curi+rng]))
    #     xp[i] = np.argmax((dR1[i,curi-20:curi+20]))
    c = c + 1


fig, ax = plt.subplots(figsize = [12,6],constrained_layout = False)
# ax.scatter(np.transpose(temp[:,curi]),Rval[:,3])
for i in range(0,c-1):
    ax.scatter(np.transpose(temp[:,curi]),Rpeak[:,i])
ax.set_xlabel('Temperature $(K)$',fontsize = 24)
ax.set_ylabel('$R_{peak}$')

In [ ]:
Kb = 1.380e-23
500 *Kb/electron_charge

In [ ]:
def temp_to_eV(temp):
    Eev = 1000*Kb*temp/electron_charge
    return Eev
def inv_temp_to_eV(E):
    temp = 0.001*electron_charge*E/Kb
    return temp


In [ ]:
fig, ax = plt.subplots(figsize = [12,6],constrained_layout = False)
# ax.scatter(np.transpose(temp[:,curi]),Rval[:,3])
ax.set_xlabel('Temperature $(K)$',fontsize = 20)
ax.set_ylabel('$R_{peak}$',fontsize = 20)
secax = ax.secondary_xaxis('top', functions=(temp_to_eV,inv_temp_to_eV))
secax.set_xlabel(r'$E_{thermal}$ $(meV)$',fontsize = 20)
for i in range(0,c-1):
    ax.scatter(np.transpose(temp[:,curi]),Rpeak[:,i], color = 'red')
    ax.scatter(np.transpose(temp[:,curi]),Rval[:,i], color = 'blue')
xx= np.linspace(34,600)
yy= 13*np.exp(-(xx-34)/200)
ax.plot(xx,yy)
xx= np.linspace(34,600)
yy= -21*np.exp(-(xx-34)/250)
ax.plot(xx,yy)

# plt.savefig(save_dir + '/plots/Temp_dependence_Ivan_peaks_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')


fig, ax = plt.subplots(figsize = [12,6],constrained_layout = False)
# ax.scatter(np.transpose(temp[:,curi]),Rval[:,3])
ax.set_xlabel('Temperature $(K)$',fontsize = 20)
ax.set_ylabel('$R_{peak}$',fontsize = 20)

secax = ax.secondary_xaxis('top', functions=(temp_to_eV,inv_temp_to_eV))
secax.set_xlabel(r'$E_{thermal}$ $(meV)$',fontsize = 20)
for i in range(0,c-1):
    ax.scatter(np.transpose(temp[:,curi]),Rpeak[:,i], color = 'red')
    ax.scatter(np.transpose(temp[:,curi]),-Rval[:,i]-2.5, color = 'blue')
xx= np.linspace(34,500)
yy= 15*np.exp(-(xx-34)/200)
ax.plot(xx,yy)

    


fig, ax = plt.subplots(figsize = [12,6],constrained_layout = False)
# ax.scatter(np.transpose(temp[:,curi]),Rval[:,3])
for i in range(0,c-1):
    ax.scatter(np.transpose(np.log(temp[:,curi])),Rpeak[:,i]/np.max(Rpeak[:,i]))
    ax.scatter(np.transpose(np.log(temp[:,curi])),Rval[:,i]/np.min(Rval[:,i]))
#     ax.scatter(np.transpose(temp[:,curi]),np.log(Rpeak[:,i]/np.max(Rpeak[:,i])), color = 'black')
#     ax.scatter(np.transpose(temp[:,curi]),np.log(Rval[:,i]/np.min(Rval[:,i])), color = 'black')

ax.set_xlabel('Temperature $(K)$',fontsize = 24)
ax.set_ylabel('$R_{peak}$')

In [ ]:
curi = np.argmin(abs(current[1,:]*1e9 - 632))
curi
Rpeak = np.zeros(N_xas)
valip = np.zeros(N_xas)
rng = 100
for i in range(0,N_xas):
    valip[i] = curi - rng + np.argmax((dR2[i,curi-rng:curi+rng]))
    
for i in range(0,N_xas):
#     Rpeak[i] = dR1[i,xp[i]]
    Rpeak[i] = np.max((dR2[i,curi-rng:curi+rng]))
#     xp[i] = np.argmax((dR1[i,curi-20:curi+20]))
    

fig, ax = plt.subplots(figsize = [12,6],constrained_layout = False)

ax.scatter(np.transpose(temp[:,curi])*1e9,Rpeak,color = 'black')
ax.set_xlabel('Temperature $(K)$',fontsize = 24)
ax.set_ylabel('$R_{peak}$')
ax.set

In [ ]:
n=11
subset = np.linspace(1,0.29,n)
subs_i = np.zeros_like(subset)
for i in range(np.size(subset)):
    subs_i[i] = np.argmin(abs(temp[:,1]- subset[i]))

matplotlib.rcParams.update({'font.size': 16})
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20) 
colors = plt.cm.coolwarm_r(np.linspace(0,1,n))

fig, ax = plt.subplots(figsize = [12,6],constrained_layout = False)
ax.set_xlabel('Current $n A$',fontsize = 24)
ax.set_ylabel('dV/dI $(\Omega)$',fontsize = 24)
# secax = ax.secondary_xaxis('top', functions=(timesXY,inv_timesXY))
# secax.set_xlabel(r'I $\times$ Rxy $(mV)$')
# subs_i = [0,10,20]
c = 0;
leg = ["" for x in range(n)]
for i in subs_i:
    ii = int(i)
    ax.plot(np.transpose(current[1,:])*1e9,np.transpose(dR1[ii,:]), linewidth = 1.5,color = colors[c])
    leg[c] ='{:.3f}K'.format(temp[ii,1])
    c = c + 1
#     ax.plot(current[1,:]*1e9,dR1[1,:], linewidth = 1.5,color = colors[i])
ax.set_xlim(-200,300)
ax.set_ylim(-5,10)
ax.legend(leg)
plt.savefig(save_dir + '/plots/dVdI_temp_dependence_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')


In [ ]:
# quick peak analysis
R = np.array([8.82, 6.92, 5.52, 4.52, 3.77, 2.18, 0.88, 0.6, 1, 0.13])/8.82
T = np.array([1.041, 0.952, 0.862, 0.768, 0.675, 0.585, 0.502, 0.437, 0.355,0.290])
T = np.flip(T)
fig, ax = plt.subplots(figsize = [12,6],constrained_layout = False)
ax.scatter(1/T,R)
fit = np.polyfit(1/T,R,1)
xf = np.array([0, 3.6])
ax.plot(xf, fit[0]*xf+fit[1])

In [ ]:
fit = np.polyfit(1/T,R,1)
fit

## dV/dI vs SG

In [ ]:
dataID = 140
dataset = qc.load_by_id(dataID)
voltage1 = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
voltage2 = dataset.get_parameter_data()['meas_voltage_K2']['meas_voltage_K2']
# voltage_AC = dataset.get_parameter_data()['meas_voltage_Lockin1']['meas_voltage_Lockin1']
current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']
# TG = dataset.get_parameter_data()['meas_voltage_K1']['appl_TG_dac10']
SG = dataset.get_parameter_data()['meas_voltage_K1']['split_gate_R']
TGS =  dataset.snapshot['station']['parameters']['top_gate']['value']


#anti clip
Nv = np.size(voltage1)
for i in range(0,Nv):
    if voltage1[i] >= 10e-3:
        voltage1[i] = np.sign(i - Nv)*10e-3
    if voltage2[i] >= 10e-3:
        voltage2[i] = np.sign(i - Nv)*10e-3


voltage1 = np.array(voltage1,dtype=np.float)
voltage2 = np.array(voltage2,dtype=np.float)


xas = SG
yas = current

N_xas = np.size(np.unique(xas))
N_yas = np.size(np.unique(yas))

if np.size(voltage1) < N_xas * N_yas:
    N_field = N_xas - 1
    voltage1 = voltage1[0:N_xas * N_yas]
    voltage2 = voltage2[0:N_xas * N_yas]    
    current = current[0:N_xas * N_yas]
    field = field[0:N_xas * N_yas]
    

voltage1 = voltage1.reshape(N_xas, N_yas)
voltage2 = voltage2.reshape(N_xas, N_yas)
xas = xas.reshape(N_xas, N_yas)
yas = yas.reshape(N_xas, N_yas)

SG = xas
current = yas

# dI = current[0,1] - current[0,0]
# dR1 = smooth_dir(voltage1,N_points = 8)/dI
# dR2 = smooth_dir(voltage2,N_points = 8)/dI


# current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']
# current_AC = 1e-9
# R_AC = voltage_AC/current_AC


# np.savez('/plots/dVdI_vs_TG_{}_{}'.format(wafer_name + sample_name, JJ))


In [ ]:
dI = current[0,1] - current[0,0]
dR1 = -smooth_dir(voltage1,N_points = 24)/dI
dR2 = smooth_dir(voltage2,N_points = 48)/dI

In [ ]:
tg = 0
i = np.argmin(abs(current[1,:] - tg))
fig, ax = plt.subplots(figsize = [8,4],constrained_layout = False)
ax.plot(SG[:,i], dR1[:,i],color = 'black')

# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
ax.set_ylim([-20,20])
ax.set_ylabel(r'dV/dI $(\Omega)$',fontsize = 22)
ax.set_xlabel(r'SG $(V)$',fontsize = 22)
# secax = ax.secondary_xaxis('top', functions=(timesXY,inv_timesXY))
# secax.set_xlabel(r'I $\times$ Rxy $(mV)$',fontsize = 22)
# ax.legend(['TG= {:.4f}'.format(TG[i,1])])
# plt.title('$V_1$ ')
# plt.savefig(save_dir + '/plots/Osc1_linecut({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')



In [ ]:
fig, ax = plt.subplots()
plt.tight_layout()
cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(1e9 * current,SG,  gaussian_filter(dR1,sigma = 0.5), cmap = cmap, shading = 'nearest'
                   , vmin=-20,vmax=20
                  )
cbar = fig.colorbar(im,ax=ax)
# x1 = [514, -1.598]
# x2 = [385, -1.599966]
# # slope:
# ax.plot([x1[0], x2[0]],[x1[1],x2[1]], linewidth = 2, color = 'black')
# slope= (x1[1] - x2[1])/((x1[0] - x2[0])*1e-9)



# ax.set_ylim(-190, 190)
# ax.set_xlim(-10, 10)
ax.set_xlabel(r'Current (nA)')
ax.set_ylabel(r'$V_{SG}$ $(V)$')
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90)
# plt.title('slope = {:.2f} (kV/A)'.format(slope * 1e-3))
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
# plt.savefig(save_dir + '/plots/dV1dI_vs_Split_Gates_regular_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()

fig, ax = plt.subplots()
plt.tight_layout()
cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(1e9 * current, SG, gaussian_filter(dR2,sigma = 0.5), cmap = cmap, shading = 'nearest'
                   , vmin=-20,vmax=20
                  )
cbar = fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)
# ax.set_xlim(-10, 10)
# x1 = [282, -1.5984]
# x2 = [-267, -1.5998]
# # slope:
# ax.plot([x1[0], x2[0]],[x1[1],x2[1]], linewidth = 2, color = 'black')
# slope= (x1[1] - x2[1])/((x1[0] - x2[0])*1e-9)

ax.set_xlabel(r'Current (nA)')
ax.set_ylabel(r'$V_{SG}$ $(V)$')
# plt.title('slope = {:.2f} (kV/A)'.format(slope * 1e-3))
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90)

# cbar.set_label(r'dV/dI ($h/e^{2}$)', rotation=90)
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)
# plt.savefig(save_dir + '/plots/dV2dI_vs_Split_Gates_regular_({})_{}.png'.format(wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()

## dV/dI vs TG Osc Slope

In [ ]:
# for samving images
save_dir = 'M:/tnw/ns/qt/2D Topo/code/personal_scripts/Mark/'
# load data
files = []
dirpath = 'M:\\tnw\\ns\\qt\\2D Topo\\samples\\InSbAs_epiAl\\M08-22-19.1_3\\22'
sample_name = dirpath.split('\\')[-1]
wafer_name = dirpath.split('\\')[-2]
db_path = dirpath + '\\data'

# r=root, d=directories, f = files
for r, d, f in os.walk(db_path):
    for file in f:
        if '.db' in file:
            files.append(os.path.join(r, file))

for f in files:
    print(f)

In [ ]:
db_path = os.path.join(db_path, 'M08-22-19.1_3_22_2021-08-26_01.db')
initialise_or_create_database_at(db_path)

In [ ]:
dataID = 64
dataset = qc.load_by_id(dataID)
voltage1 = dataset.get_parameter_data()['meas_voltage_K1']['meas_voltage_K1']
voltage2 = dataset.get_parameter_data()['meas_voltage_K2']['meas_voltage_K2']
# voltage_AC = dataset.get_parameter_data()['meas_voltage_Lockin1']['meas_voltage_Lockin1']
current = dataset.get_parameter_data()['meas_voltage_K1']['appl_current']
TG = dataset.get_parameter_data()['meas_voltage_K1']['delta_top_gate']


TG_start = dataset.snapshot['station']['parameters']['top_gate']['value']

#anti clip
Nv = np.size(voltage1)
for i in range(0,Nv):
    if voltage1[i] >= 10e-3:
        voltage1[i] = np.sign(i - Nv)*10e-3
    if voltage2[i] >= 10e-3:
        voltage2[i] = np.sign(i - Nv)*10e-3


voltage1 = np.array(voltage1,dtype=np.float)
voltage2 = np.array(voltage2,dtype=np.float)


xas = TG
yas = current

N_xas = np.size(np.unique(xas))
N_yas = np.size(np.unique(yas))

if np.size(voltage1) < N_xas * N_yas:
    N_field = N_xas - 1
    voltage1 = voltage1[0:N_xas * N_yas]
    voltage2 = voltage2[0:N_xas * N_yas]    
    current = current[0:N_xas * N_yas]
    field = field[0:N_xas * N_yas]
    

voltage1 = voltage1.reshape(N_xas, N_yas)
voltage2 = voltage2.reshape(N_xas, N_yas)
xas = xas.reshape(N_xas, N_yas)
yas = yas.reshape(N_xas, N_yas)

TG = xas
current = yas

dI = current[0,1] - current[0,0]
dR1 = smooth_dir(voltage1,N_points = 6)/dI
dR2 = smooth_dir(voltage2,N_points = 6)/dI


# current_AC = dataset.snapshot['station']['parameters']['appl_current_AC']['value']
# current_AC = 1e-9
# R_AC = voltage_AC/current_AC


# np.savez('/plots/dVdI_vs_TG_{}_{}'.format(wafer_name + sample_name, JJ))


In [ ]:
HB='HB2'

In [ ]:

fig, ax = plt.subplots()

cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(1e9 * current, TG_start + TG, dR1, cmap = cmap, shading = 'nearest'
                   , vmin=-200
                   ,vmax=200
                  )
cbar = fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)
# ax.set_xlim(-10, 10)
ax.set_xlabel(r'Current (nA)')
ax.set_ylabel(r'V gate (V)')
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90)
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)



#for exp 62
# x1 = [261,-1.2003756]
# x2 = [202.4, -1.2019034]
# nu = 4

#for exp 63
# x1 = [-59.4,-1.150141]
# x2 = [-324.2, -1.153879]
# nu = 5

# for exp 64
x1 = [350.3, -1.013831]
x2 = [176.2,-1.015430]
nu = 6

# secax = ax.secondary_xaxis('top', functions=(timesXY,inv_timesXY))
# secax.set_xlabel(r'I $\times$ Rxy $(mV)$')

# ($\\nu = {}$)
ax.plot([x1[0], x2[0]],[x1[1],x2[1]], linewidth = 3, color = 'black')
slope= (x1[1] - x2[1])/((x1[0] - x2[0])*1e-9)

plt.title('$V_1$ \n slope = {:.0f} $(V/A)$'.format(slope))
plt.savefig(save_dir + '/plots/slope/dV1dI_vs_TG_nu={}_({})_{}.png'.format(nu,wafer_name + '_' + sample_name, HB),bbox_inches='tight')

plt.show()

In [ ]:
slope

In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap('seismic')
im = ax.pcolormesh(1e9 * current,TG_start +  TG, dR2, cmap = cmap, shading = 'nearest'
                   , vmin=-100,vmax=100
                  )
cbar = fig.colorbar(im,ax=ax)
# ax.set_ylim(-190, 190)
# ax.set_xlim(-10, 10)
ax.set_xlabel(r'Current (nA)')
ax.set_ylabel(r'V gate (V)')
cbar.set_label(r'dV/dI ($\Omega$)', rotation=90)
# plt.title(wafer_name +  ' ' + sample_name + ' ' + HB)



# #for exp 62
# x1 = [180.3, -1.200155]
# x2 = [115.4, -1.2018672]
# nu = 4

#for exp 63
# x1 = [327.4,-1.150119]
# x2 = [183.3,-1.152250]
# nu = 5

# #for exp 64
x1 = [323.7, -1.013425]
x2 = [152, -1.015161]
nu = 6
# secax = ax.secondary_xaxis('top', functions=(timesXY,inv_timesXY))
# secax.set_xlabel(r'I $\times$ Rxy $(mV)$')


ax.plot([x1[0], x2[0]],[x1[1],x2[1]], linewidth = 3, color = 'black')
slope= (x1[1] - x2[1])/((x1[0] - x2[0])*1e-9)
# ($\\nu = {}$)
plt.title('$V_2$ \n slope = {:.0f} $(V/A)$'.format(slope))
plt.savefig(save_dir + '/plots/slope/dV2dI_vs_TG_nu={}_({})_{}.png'.format(nu,wafer_name + '_' + sample_name, HB),bbox_inches='tight')
plt.show()